In [13]:
import numpy as np
import pandas as pd
import re
import os


In [8]:
#these are all the data files we will be using, the .csvs are not in the repository for space reasons
#but the files are available on the google drive
price= pd.read_csv('NADAC_as_of_2017-06-07.csv',dtype={'NDC': str})
product= pd.read_csv('product.csv')
package=pd.read_csv('package.csv')
NADAC_Comparison = pd.read_csv('NADAC_Comparison.csv',dtype={'NDC':str})
price_series = pd.read_csv('NADAC__National_Average_Drug_Acquisition_Cost_.csv')



In [12]:
#here are various data cleaning operations we must do to analyze the data

#make percent change a float
NADAC_Comparison['Percent Change']=NADAC_Comparison['Percent Change'].apply(lambda x: x[:-1])
NADAC_Comparison['Percent Change']=NADAC_Comparison['Percent Change'].apply(pd.to_numeric)

#change the NDC so it matches the NDC in the other .csv


TypeError: 'float' object has no attribute '__getitem__'

In [14]:
price['NDC10']=price['NDC'].apply(lambda x: x[0:5]+x[6:])  #de18.founded=de18.founded.apply(lambda x: x[8:])

#merge two csv's together
FDA = pd.merge(package,product,how='inner',on='PRODUCTNDC')

#remove the - from the NDC to match the other NDC
import re
def func(x):                                       # replace the - for ''in column:NDCPACKAGECODE for file FDA
    return re.sub('[-]',"", str(x))
FDA['NDC'] = FDA['NDCPACKAGECODE'].apply(func)

In [15]:
NADAC_Comparison.head(1)
NADAC_Comparison.groupby('Primary Reason').agg(["mean","count"])['Percent Change']

,mean,count
Primary Reason,,
Effective Date Change,0.000000,1
Help Desk,138.755338,1999
Rate Group Change,-6.993731,67
Survey Rate,0.066226,815422
WAC Adjustment,10.691823,13091


In [21]:
FDA['NDC']

0         0002120030
1         0002120050
2         0002140701
3         0002143361
4         0002143380
5         0002143461
6         0002143480
7         0002144509
8         0002144511
9         0002144527
10        0002144561
11        0002197561
12        0002197590
13        0002300475
14        0002322707
15        0002322730
16        0002322807
17        0002322830
18        0002322907
19        0002322930
20        0002323030
21        0002323130
22        0002323130
23        0002803101
24        0002803101
25        0002323230
26        0002323330
27        0002323430
28        0002323560
29        0002323560
             ...    
223250    9920727601
223251    9920727675
223252    9920729002
223253    9920729102
223254    9920729202
223255    9920730002
223256    9920730030
223257    9920730060
223258    9920746301
223259    9920746302
223260    9920746330
223261    9920746401
223262    9920746402
223263    9920746430
223264    9920746501
223265    9920746502
223266    992

In [22]:
priceInfo = pd.merge(FDA,price,right_on='NDC10',left_on='NDC')



In [23]:
#priceInfo = priceInfo.loc[:,['DOSAGEFORMNAME'] + ['ROUTENAME'] + ['STARTMARKETINGDATE'] + ['LABELERNAME']  + ['PHARM_CLASSES'] + ['DEASCHEDULE'] + ['NADAC_Per_Unit']]

priceInfo = priceInfo.loc[:,['DOSAGEFORMNAME'] + ['ROUTENAME'] + ['STARTMARKETINGDATE']  + ['PHARM_CLASSES'] + ['NADAC_Per_Unit']]

def Dummify(column):
    column_values = pd.get_dummies(priceInfo[column], prefix=column, prefix_sep='__')
    return pd.concat([priceInfo.drop(column,axis=1),column_values],axis=1)

priceInfo = Dummify('DOSAGEFORMNAME')
priceInfo = Dummify('ROUTENAME')
#priceInfo = Dummify('LABELERNAME')
priceInfo = Dummify('PHARM_CLASSES')
#priceInfo = Dummify('DEASCHEDULE')



In [25]:
priceInfo.head(1)

,STARTMARKETINGDATE,NADAC_Per_Unit,"DOSAGEFORMNAME__AEROSOL, FOAM","DOSAGEFORMNAME__AEROSOL, METERED","DOSAGEFORMNAME__AEROSOL, POWDER","DOSAGEFORMNAME__AEROSOL, SPRAY",DOSAGEFORMNAME__CAPSULE,"DOSAGEFORMNAME__CAPSULE, COATED PELLETS","DOSAGEFORMNAME__CAPSULE, COATED, EXTENDED RELEASE","DOSAGEFORMNAME__CAPSULE, DELAYED RELEASE",...,"PHARM_CLASSES__Vitamin D [Chemical/Ingredient],Vitamin D [EPC],Vitamin B6 Analog [EPC],Vitamin B 6 [Chemical/Ingredient],Analogs/Derivatives [Chemical/Ingredient],Vitamin B 12 [Chemical/Ingredient],Vitamin B12 [EPC],Calculi Dissolution Agent [EPC],Magnesium Ion Exchange Activity [MoA],Osmotic Laxative [EPC],Osmotic Activity [MoA],Inhibition Small Intestine Fluid/Electrolyte Absorption [PE],Increased Large Intestinal Motility [PE],Stimulation Large Intestine Fluid/Electrolyte Secretion [PE],Inhibition Large Intestine Fluid/Electrolyte Absorption [PE]","PHARM_CLASSES__Vitamin K Antagonist [EPC],Vitamin K Inhibitors [MoA]","PHARM_CLASSES__Xanthine Oxidase Inhibitor [EPC],Xanthine Oxidase Inhibitors [MoA]","PHARM_CLASSES__alpha Glucosidase Inhibitors [MoA],alpha-Glucosidase Inhibitor [EPC]","PHARM_CLASSES__gamma-Aminobutyric Acid A Receptor Agonist [EPC],GABA A Agonists [MoA],Central Nervous System Depression [PE]","PHARM_CLASSES__gamma-Aminobutyric Acid-ergic Agonist [EPC],GABA A Agonists [MoA],Pyridines [Chemical/Ingredient],Central Nervous System Depression [PE]","PHARM_CLASSES__l-Thyroxine [EPC],Thyroxine [Chemical/Ingredient]","PHARM_CLASSES__l-Thyroxine [EPC],Thyroxine [Chemical/Ingredient],l-Triiodothyronine [EPC],Triiodothyronine [Chemical/Ingredient]","PHARM_CLASSES__l-Triiodothyronine [EPC],Triiodothyronine [Chemical/Ingredient]","PHARM_CLASSES__mu-Opioid Receptor Agonist [EPC],Opioid mu-Receptor Agonists [MoA]"
0,19590528.0,0.02878,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
from sklearn import linear_model
logit_1 = linear_model.LogisticRegression()

In [27]:
def f(row):
    if (row['NADAC_Per_Unit'] > 10):
        return 0
    else:
        return 1
priceInfo['priceType'] = priceInfo.apply(f, axis=1)

priceInfo = ['priceType'].head()

0    1
1    1
2    1
3    0
4    0
Name: priceType, dtype: int64

In [ ]:
priceInfo = priceInfo.drop("NADAC_Per_Unit",axis=1)

In [31]:
logit_1.set_params(C=1e4)
y = priceInfo["priceType"]
x = priceInfo.loc[:, priceInfo.columns != 'priceType']
logit_1.fit(x, y)


LogisticRegression(C=10000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [33]:
print(logit_1.score(x, y))

0.940238131583


In [49]:
#this is to generate a list of all the averages of all the values to pass in as a default value if the user doesn't specify
#a value

cols = list(priceInfo.columns.values)
means = []
for e in cols:
    means.append(priceInfo[e].mean())
means
inpt = zip(cols,means)
cols

['STARTMARKETINGDATE',
 'DOSAGEFORMNAME__AEROSOL, FOAM',
 'DOSAGEFORMNAME__AEROSOL, METERED',
 'DOSAGEFORMNAME__AEROSOL, POWDER',
 'DOSAGEFORMNAME__AEROSOL, SPRAY',
 'DOSAGEFORMNAME__CAPSULE',
 'DOSAGEFORMNAME__CAPSULE, COATED PELLETS',
 'DOSAGEFORMNAME__CAPSULE, COATED, EXTENDED RELEASE',
 'DOSAGEFORMNAME__CAPSULE, DELAYED RELEASE',
 'DOSAGEFORMNAME__CAPSULE, DELAYED RELEASE PELLETS',
 'DOSAGEFORMNAME__CAPSULE, EXTENDED RELEASE',
 'DOSAGEFORMNAME__CAPSULE, GELATIN COATED',
 'DOSAGEFORMNAME__CAPSULE, LIQUID FILLED',
 'DOSAGEFORMNAME__CONCENTRATE',
 'DOSAGEFORMNAME__CREAM',
 'DOSAGEFORMNAME__CREAM, AUGMENTED',
 'DOSAGEFORMNAME__ELIXIR',
 'DOSAGEFORMNAME__EMULSION',
 'DOSAGEFORMNAME__ENEMA',
 'DOSAGEFORMNAME__FILM',
 'DOSAGEFORMNAME__FILM, EXTENDED RELEASE',
 'DOSAGEFORMNAME__FOR SOLUTION',
 'DOSAGEFORMNAME__FOR SUSPENSION',
 'DOSAGEFORMNAME__GEL',
 'DOSAGEFORMNAME__GEL, DENTIFRICE',
 'DOSAGEFORMNAME__GRANULE',
 'DOSAGEFORMNAME__GRANULE, FOR SOLUTION',
 'DOSAGEFORMNAME__GRANULE, FOR SUSP

In [51]:
#this function should take arguments for each of the parameters to the logistic regression
#with the averages being the default and return a list to be used to make predictions
#using the logistic models, right now it gives a syntax error
def prediction(STARTMARKETINGDATE=20096656.36437185,DOSAGEFORMNAME__AEROSOLFOAM=0.0012211876049458099,DOSAGEFORMNAME__AEROSOLMETERED=0.00076324225309113108,DOSAGEFORMNAME__AEROSOLPOWDER=7.6324225309113119e05,DOSAGEFORMNAME__AEROSOLSPRAY=0.00038162112654556554,DOSAGEFORMNAME__CAPSULE=0.083117081361624184,DOSAGEFORMNAME__CAPSULECOATEDPELLETS=0.00053426957716379178,DOSAGEFORMNAME__CAPSULECOATEDEXTENDEDRELEASE=0.0042741566173103342,DOSAGEFORMNAME__CAPSULEDELAYEDRELEASE=0.016333384216150205,DOSAGEFORMNAME__CAPSULEDELAYEDRELEASEPELLETS=0.0016791329568004885,DOSAGEFORMNAME__CAPSULEEXTENDEDRELEASE=0.026331857731644023,DOSAGEFORMNAME__CAPSULEGELATINCOATED=0.0024423752098916198,DOSAGEFORMNAME__CAPSULELIQUIDFILLED=0.0077087467562204246,DOSAGEFORMNAME__CONCENTRATE=0.00030529690123645248,DOSAGEFORMNAME__CREAM=0.019462677453823844,DOSAGEFORMNAME__CREAMAUGMENTED=7.6324225309113119e05,DOSAGEFORMNAME__ELIXIR=0.00022897267592733933,DOSAGEFORMNAME__EMULSION=0.00045794535185467866,DOSAGEFORMNAME__ENEMA=0.00076324225309113108,DOSAGEFORMNAME__FILM=0.00053426957716379178,DOSAGEFORMNAME__FILMEXTENDEDRELEASE=0.00015264845061822624,DOSAGEFORMNAME__FORSOLUTION=0.00038162112654556554,DOSAGEFORMNAME__FORSUSPENSION=0.00076324225309113108,DOSAGEFORMNAME__GEL=0.0070981529537475193,DOSAGEFORMNAME__GELDENTIFRICE=0.00015264845061822624,DOSAGEFORMNAME__GRANULE=0.00053426957716379178,DOSAGEFORMNAME__GRANULEFORSOLUTION=7.6324225309113119e05,DOSAGEFORMNAME__GRANULEFORSUSPENSION=0.00022897267592733933,DOSAGEFORMNAME__GUMCHEWING=0.0025186994352007328,DOSAGEFORMNAME__INJECTION=0.0045794535185467869,DOSAGEFORMNAME__INJECTIONPOWDERFORSOLUTION=0.00061059380247290495,DOSAGEFORMNAME__INJECTIONPOWDERLYOPHILIZEDFORSOLUTION=0.001373836055564036,DOSAGEFORMNAME__INJECTIONSOLUTION=0.0066402076018928407,DOSAGEFORMNAME__INJECTIONSUSPENSION=0.00030529690123645248,DOSAGEFORMNAME__INJECTIONSUSPENSIONEXTENDEDRELEASE=0.00038162112654556554,DOSAGEFORMNAME__JELLY=0.00068691802778201802,DOSAGEFORMNAME__KIT=0.015264845061822623,DOSAGEFORMNAME__LIQUID=0.011067012669821401,DOSAGEFORMNAME__LOTION=0.0032056174629827507,DOSAGEFORMNAME__LOZENGE=0.0019844298580369408,DOSAGEFORMNAME__OIL=0.00083956647840024426,DOSAGEFORMNAME__OINTMENT=0.011448633796366967,DOSAGEFORMNAME__PASTE=7.6324225309113119e05,DOSAGEFORMNAME__PATCH=0.00343459013891009,DOSAGEFORMNAME__PATCHEXTENDEDRELEASE=0.0035109143642192034,DOSAGEFORMNAME__PELLET=7.6324225309113119e05,DOSAGEFORMNAME__POWDER=0.0012211876049458099,DOSAGEFORMNAME__POWDERFORSOLUTION=0.0030529690123645243,DOSAGEFORMNAME__POWDERFORSUSPENSION=0.010151121966112044,DOSAGEFORMNAME__POWDERMETERED=7.6324225309113119e05,DOSAGEFORMNAME__RINSE=0.00038162112654556554,DOSAGEFORMNAME__SHAMPOO=0.00091589070370935732,DOSAGEFORMNAME__SHAMPOOSUSPENSION=0.00015264845061822624,DOSAGEFORMNAME__SOLUTION=0.027858342237826288,DOSAGEFORMNAME__SOLUTIONCONCENTRATE=0.00022897267592733933,DOSAGEFORMNAME__SOLUTIONGELFORMINGDROPS=7.6324225309113119e05,DOSAGEFORMNAME__SOLUTIONGELFORMINGEXTENDEDRELEASE=0.00015264845061822624,DOSAGEFORMNAME__SOLUTIONDROPS=0.012593497176003663,DOSAGEFORMNAME__SPRAY=0.0027476721111280721,DOSAGEFORMNAME__SPRAYMETERED=0.0017554571821096017,DOSAGEFORMNAME__SUPPOSITORY=0.0012975118302549229,DOSAGEFORMNAME__SUSPENSION=0.011372309571057854,DOSAGEFORMNAME__SUSPENSIONEXTENDEDRELEASE=0.0019844298580369408,DOSAGEFORMNAME__SUSPENSIONDROPS=0.0025186994352007328,DOSAGEFORMNAME__SWAB=7.6324225309113119e05,DOSAGEFORMNAME__SYRUP=0.0031292932376736377,DOSAGEFORMNAME__TABLET=0.34979392459166542,DOSAGEFORMNAME__TABLETCHEWABLE=0.010685391543275836,DOSAGEFORMNAME__TABLETCOATED=0.014806899709967944,DOSAGEFORMNAME__TABLETDELAYEDRELEASE=0.012135551824148985,DOSAGEFORMNAME__TABLETEFFERVESCENT=0.00068691802778201802,DOSAGEFORMNAME__TABLETEXTENDEDRELEASE=0.040909784765684631,DOSAGEFORMNAME__TABLETFILMCOATED=0.20775454129140589,DOSAGEFORMNAME__TABLETFILMCOATEDEXTENDEDRELEASE=0.024576400549534423,DOSAGEFORMNAME__TABLETFORSUSPENSION=0.00015264845061822624,DOSAGEFORMNAME__TABLETMULTILAYER=7.6324225309113119e05,DOSAGEFORMNAME__TABLETMULTILAYEREXTENDEDRELEASE=7.6324225309113119e05,DOSAGEFORMNAME__TABLETORALLYDISINTEGRATING=0.011753930697603419,DOSAGEFORMNAME__TABLETORALLYDISINTEGRATINGDELAYEDRELEASE=0.00015264845061822624,DOSAGEFORMNAME__TABLETSUGARCOATED=0.00053426957716379178,DOSAGEFORMNAME__TINCTURE=7.6324225309113119e05,ROUTENAME__AURICULAROTIC=0.0017554571821096017,ROUTENAME__BUCCAL=0.00053426957716379178,ROUTENAME__BUCCAL;SUBLINGUAL=0.00022897267592733933,ROUTENAME__CUTANEOUS=0.00030529690123645248,ROUTENAME__DENTAL=0.00015264845061822624,ROUTENAME__ENTERAL=7.6324225309113119e05,ROUTENAME__INTRAARTERIAL;INTRAMUSCULAR;INTRATHECAL;INTRAVENOUS=0.00030529690123645248,ROUTENAME__INTRAARTERIAL;INTRAMUSCULAR;INTRAVENOUS=0.00022897267592733933,ROUTENAME__INTRAARTICULAR;INTRALESIONAL;INTRAMUSCULAR;INTRAVENOUS;SOFTTISSUE=0.00022897267592733933,ROUTENAME__INTRAARTICULAR;INTRALESIONAL;INTRAMUSCULAR;SOFTTISSUE=7.6324225309113119e05,ROUTENAME__INTRADERMAL=0.00015264845061822624,ROUTENAME__INTRALESIONAL;INTRAMUSCULAR;INTRASYNOVIAL;SOFTTISSUE=7.6324225309113119e05,ROUTENAME__INTRAMUSCULAR=0.0026713478858189591,ROUTENAME__INTRAMUSCULAR;INTRAVENOUS=0.0027476721111280721,ROUTENAME__INTRAMUSCULAR;INTRAVENOUS;SUBCUTANEOUS=0.00053426957716379178,ROUTENAME__INTRAMUSCULAR;SUBCUTANEOUS=0.00022897267592733933,ROUTENAME__INTRAOCULAR=0.00038162112654556554,ROUTENAME__INTRAVENOUS=0.0016028087314913753,ROUTENAME__INTRAVENOUS;SUBCUTANEOUS=0.0014501602808731492,ROUTENAME__NASAL=0.0038162112654556556,ROUTENAME__OPHTHALMIC=0.019310029003205619,ROUTENAME__OPHTHALMIC;TOPICAL=0.00022897267592733933,ROUTENAME__ORAL=0.8794840482369104,ROUTENAME__ORAL;ORAL=7.6324225309113119e05,ROUTENAME__ORAL;RECTAL=0.00045794535185467866,ROUTENAME__ORAL;RESPIRATORYINHALATION=0.00022897267592733933,ROUTENAME__ORAL;SUBLINGUAL=7.6324225309113119e05,ROUTENAME__ORAL;TOPICAL=0.00022897267592733933,ROUTENAME__ORAL;TRANSMUCOSAL=0.00015264845061822624,ROUTENAME__PERINEURAL=7.6324225309113119e05,ROUTENAME__RECTAL=0.0023660509845825064,ROUTENAME__RECTAL;TOPICAL=0.00022897267592733933,ROUTENAME__RESPIRATORYINHALATION=0.0027476721111280721,ROUTENAME__SUBCUTANEOUS=0.0040451839413829949,ROUTENAME__SUBLINGUAL=0.0014501602808731492,ROUTENAME__TOPICAL=0.049839719126850861,ROUTENAME__TRANSDERMAL=0.0073271256296748586,ROUTENAME__VAGINAL=0.0016028087314913753,PHARM_CLASSES__5LipoxygenaseInhibitorEPC5LipoxygenaseInhibitorsMoADecreasedLeukotrieneProductionPE=7.6324225309113119e05,PHARM_CLASSES__5alphaReductaseInhibitorEPC5alphaReductaseInhibitorsMoA=0.0043504808426194476,PHARM_CLASSES__5alphaReductaseInhibitorEPC5alphaReductaseInhibitorsMoAAdrenergicalphaAntagonistsMoAalphaAdrenergicBlockerEPC=0.00030529690123645248,PHARM_CLASSES__AcetylAldehydeDehydrogenaseInhibitorsMoAAldehydeDehydrogenaseInhibitorEPC=0.00038162112654556554,PHARM_CLASSES__AcidifyingActivityMoACalculiDissolutionAgentEPCAnticoagulantEPCDecreasedCoagulationFactorActivityPECalciumChelatingActivityMoA=0.00038162112654556554,PHARM_CLASSES__AdrenergicAgonistsMoAAdrenergicReceptorAgonistEPCDecreasedHistamineReleasePEHistamineH1ReceptorAntagonistsMoAHistamine1ReceptorInhibitorEPC=0.00015264845061822624,PHARM_CLASSES__AdrenergicalphaAgonistsMoAAdrenergicbetaAgonistsMoAalphaAdrenergicAgonistEPCbetaAdrenergicAgonistEPCCatecholamineEPCCatecholaminesChemicalIngredient=0.00061059380247290495,PHARM_CLASSES__AdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPC=0.0019081056327278278,PHARM_CLASSES__AdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPCUncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoA=0.00045794535185467866,PHARM_CLASSES__AdrenergicalphaAntagonistsMoAAdrenergicbeta1AntagonistsMoAAdrenergicbeta2AntagonistsMoAalphaAdrenergicBlockerEPCbetaAdrenergicBlockerEPC=0.0061822622500381621,PHARM_CLASSES__AdrenergicalphaAntagonistsMoAalphaAdrenergicBlockerEPC=0.0061059380247290487,PHARM_CLASSES__Adrenergicalpha2AgonistsMoACentralalpha2AdrenergicAgonistEPC=0.0064875591512746147,PHARM_CLASSES__AdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPC=0.019996947030987637,PHARM_CLASSES__AdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__Adrenergicbeta2AgonistsMoAbeta2AdrenergicAgonistEPC=0.0024423752098916198,PHARM_CLASSES__Adrenergicbeta2AgonistsMoAbeta2AdrenergicAgonistEPCAnticholinergicEPCCholinergicAntagonistsMoA=0.00015264845061822624,PHARM_CLASSES__AldosteroneAntagonistEPCAldosteroneAntagonistsMoA=0.0029766447870554114,PHARM_CLASSES__AldosteroneAntagonistEPCAldosteroneAntagonistsMoAIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__AlkylatingActivityMoAAlkylatingDrugEPC=0.00091589070370935732,PHARM_CLASSES__AllylamineChemicalIngredientAllylamineAntifungalEPC=0.001373836055564036,PHARM_CLASSES__AluminumComplexEPCOrganometallicCompoundsChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientAntiarrhythmicEPCLocalAnesthesiaPE=0.0015264845061822622,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientAntiarrhythmicEPCLocalAnesthesiaPEAmideLocalAnestheticEPCAmidesChemicalIngredientLocalAnesthesiaPE=7.6324225309113119e05,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientAntiarrhythmicEPCLocalAnesthesiaPECorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00038162112654556554,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientLocalAnesthesiaPE=7.6324225309113119e05,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient=0.0018317814074187146,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00068691802778201802,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00061059380247290495,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDecreasedCellWallSynthesis&amp;RepairPECorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00022897267592733933,PHARM_CLASSES__AminoketoneEPCDopamineUptakeInhibitorsMoAIncreasedDopamineActivityPEIncreasedNorepinephrineActivityPENorepinephrineUptakeInhibitorsMoA=0.0058006411234925968,PHARM_CLASSES__AminosalicylateEPCAminosalicylicAcidsChemicalIngredient=0.00099221492901847039,PHARM_CLASSES__AmylinAgonistsMoAAmylinAnalogEPC=7.6324225309113119e05,PHARM_CLASSES__AndrogenReceptorAntagonistsMoAAndrogenReceptorInhibitorEPC=0.00053426957716379178,PHARM_CLASSES__AndrogenEPCAndrogenReceptorAgonistsMoAAndrostanesChemicalIngredient=0.00091589070370935732,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC=0.022362998015570141,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCCalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCCalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.0019081056327278278,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.01030377041673027,PHARM_CLASSES__Angiotensin2Type1ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCDecreasedBloodPressurePE=0.00022897267592733933,PHARM_CLASSES__Angiotensin2Type1ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCDecreasedBloodPressurePEIncreasedDiuresisPEThiazidelikeDiureticEPC=0.00015264845061822624,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoA=0.018699435200732712,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoACalciumChannelAntagonistsMoACalciumChannelBlockerEPCCytochromeP4503A4InhibitorsMoACytochromeP4503AInhibitorsMoAPGlycoproteinInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoADecreasedBloodPressurePE=0.0046557777438558995,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoADecreasedBloodPressurePEIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.00030529690123645248,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoAIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.0022897267592733935,PHARM_CLASSES__AnticoagulantEPCHeparinChemicalIngredientUnfractionatedHeparinEPC=0.00068691802778201802,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPE=0.016562356892077544,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPECytochromeP4501A2InducersMoACytochromeP4502B6InducersMoACytochromeP4502C8InducersMoACytochromeP4502C19InducersMoACytochromeP4502D6InducersMoACytochromeP4503AInducersMoACytochromeP4502C9InducersMoA=0.0012211876049458099,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPEMoodStabilizerEPC=0.013356739429094794,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPESulfonamidesChemicalIngredientCarbonicAnhydraseInhibitorsMoAPGlycoproteinInhibitorsMoA=0.0016028087314913753,PHARM_CLASSES__AntiarrhythmicEPC=0.0035872385895283163,PHARM_CLASSES__AntiarrhythmicEPCCardiacRhythmAlterationPEAdrenergicbetaAntagonistsMoA=0.00015264845061822624,PHARM_CLASSES__AntiarrhythmicEPCCytochromeP4503A4InhibitorsMoAPGlycoproteinInhibitorsMoACytochromeP4501A2InhibitorsMoACytochromeP4502C9InhibitorsMoACytochromeP4502D6InhibitorsMoA=0.00061059380247290495,PHARM_CLASSES__AnticholinergicEPCAntihistamineEPCCholinergicAntagonistsMoAHistamineReceptorAntagonistsMoA=0.002823996336437185,PHARM_CLASSES__AnticholinergicEPCCholinergicAntagonistsMoA=0.00343459013891009,PHARM_CLASSES__AnticholinergicEPCCholinergicAntagonistsMoAAdrenergicbeta2AgonistsMoAbeta2AdrenergicAgonistEPC=0.00038162112654556554,PHARM_CLASSES__AnticholinergicEPCCholinergicAntagonistsMoACholinergicMuscarinicAntagonistEPCCholinergicMuscarinicAntagonistsMoA=0.00030529690123645248,PHARM_CLASSES__AntidoteEPCAntidoteforAcetaminophenOverdoseEPCDecreasedRespiratorySecretionViscosityPEIncreasedGlutathioneConcentrationPEMucolyticEPCReductionActivityMoA=0.00022897267592733933,PHARM_CLASSES__AntiemeticEPCEmesisSuppressionPE=0.0019081056327278278,PHARM_CLASSES__AntifibrinolyticAgentEPCDecreasedFibrinolysisPE=0.00030529690123645248,PHARM_CLASSES__AntihelminthicEPC=7.6324225309113119e05,PHARM_CLASSES__AntihelminthicEPCCytochromeP4501AInducersMoA=0.00015264845061822624,PHARM_CLASSES__AntihistamineEPCHistamineReceptorAntagonistsMoA=0.0058769653488017094,PHARM_CLASSES__AntihistamineEPCHistamineReceptorAntagonistsMoAVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__AntimalarialEPC=0.00045794535185467866,PHARM_CLASSES__AntimalarialEPCAntiprotozoalEPC=0.00015264845061822624,PHARM_CLASSES__AntimalarialEPCAntiprotozoalEPCAntimalarialEPCDihydrofolateReductaseInhibitorsMoA=0.00045794535185467866,PHARM_CLASSES__AntimetaboliteImmunosuppressantEPC=0.0016028087314913753,PHARM_CLASSES__AntimetaboliteEPCUreaChemicalIngredient=0.00015264845061822624,PHARM_CLASSES__AntimycobacterialEPC=0.0012211876049458099,PHARM_CLASSES__AntiparasiticEPCPediculicideEPC=0.00022897267592733933,PHARM_CLASSES__AntiprotozoalEPC=0.00022897267592733933,PHARM_CLASSES__AntirheumaticAgentEPC=0.00053426957716379178,PHARM_CLASSES__AntirheumaticAgentEPCAntimalarialEPC=0.00099221492901847039,PHARM_CLASSES__AppetiteSuppressionPEIncreasedSympatheticActivityPESympathomimeticAmineAnorecticEPC=0.0047321019691650128,PHARM_CLASSES__AppetiteSuppressionPEIncreasedSympatheticActivityPESympathomimeticAmineAnorecticEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPECytochromeP4503A4InducersMoACytochromeP4502C19InhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__AromataseInhibitorEPCAromataseInhibitorsMoA=0.0015264845061822622,PHARM_CLASSES__AromaticAminoAcidDecarboxylationInhibitorEPCDOPADecarboxylaseInhibitorsMoAAromaticAminoAcidEPCAminoAcidsAromaticChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__AromaticAminoAcidEPCAminoAcidsAromaticChemicalIngredient=0.0031292932376736377,PHARM_CLASSES__AromaticAminoAcidEPCAminoAcidsAromaticChemicalIngredientCatecholOMethyltransferaseInhibitorsMoACatecholOMethyltransferaseInhibitorEPC=0.00091589070370935732,PHARM_CLASSES__ArteriolarVasodilationPEArteriolarVasodilatorEPC=0.0049610746450923521,PHARM_CLASSES__ArteriolarVasodilationPEArteriolarVasodilatorEPCNitrateVasodilatorEPCNitratesChemicalIngredientVasodilationPE=7.6324225309113119e05,PHARM_CLASSES__AtypicalAntipsychoticEPC=0.040680812089757289,PHARM_CLASSES__AtypicalAntipsychoticEPCSerotoninReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoA=0.00038162112654556554,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredient=0.0012211876049458099,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00045794535185467866,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4502C19InhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4502C19InhibitorsMoACytochromeP4503A4InhibitorsMoACytochromeP4502C9InhibitorsMoA=0.0031292932376736377,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4502C19InhibitorsMoACytochromeP4502C9InhibitorsMoA=0.00083956647840024426,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4503A5InhibitorsMoAPGlycoproteinInhibitorsMoA=0.00022897267592733933,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4503A4InhibitorsMoAPGlycoproteinInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredient=0.00045794535185467866,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredientCentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredient=0.00091589070370935732,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredientCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredient=0.00030529690123645248,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredientCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredientFullOpioidAgonistsMoAOpioidAgonistEPC=7.6324225309113119e05,PHARM_CLASSES__BenzodiazepineEPCBenzodiazepinesChemicalIngredient=0.010609067317966722,PHARM_CLASSES__BenzodiazepineEPCBenzodiazepinesChemicalIngredientAnticholinergicEPCDecreasedParasympatheticAcetylcholineActivityPEGIMotilityAlterationPEDigestiveGISystemActivityAlterationPECholinergicAntagonistsMoA=0.00015264845061822624,PHARM_CLASSES__BenzodiazepineEPCBenzodiazepinesChemicalIngredientCytochromeP4502D6InhibitorsMoA=0.00022897267592733933,PHARM_CLASSES__BenzothiazoleEPCBenzothiazolesChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__BiguanideEPCBiguanidesChemicalIngredient=0.013662036330331247,PHARM_CLASSES__BiguanideEPCBiguanidesChemicalIngredientDipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPC=7.6324225309113119e05,PHARM_CLASSES__BiguanideEPCBiguanidesChemicalIngredientPeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredient=0.00030529690123645248,PHARM_CLASSES__BileAcidSequestrantEPCBileacidBindingActivityMoA=0.00053426957716379178,PHARM_CLASSES__BileAcidEPCBileAcidsandSaltsChemicalIngredient=0.0010685391543275836,PHARM_CLASSES__BismuthChemicalIngredientBismuthEPCNitroimidazoleAntimicrobialEPCNitroimidazolesChemicalIngredientTetracyclineclassAntimicrobialEPCTetracyclinesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__BisphosphonateEPCDiphosphonatesChemicalIngredient=0.0031292932376736377,PHARM_CLASSES__BloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredient=0.00076324225309113108,PHARM_CLASSES__BloodViscosityReducerEPCHematologicActivityAlterationPE=0.00022897267592733933,PHARM_CLASSES__CCR5CoreceptorAntagonistEPCChemokineCoreceptor5AntagonistsMoA=0.00015264845061822624,PHARM_CLASSES__CalcineurinInhibitorImmunosuppressantEPCCalcineurinInhibitorsMoA=0.0018317814074187146,PHARM_CLASSES__CalcineurinInhibitorImmunosuppressantEPCCalcineurinInhibitorsMoACytochromeP4503A4InhibitorsMoAPGlycoproteinInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__CalcitoninChemicalIngredientCalcitoninEPC=7.6324225309113119e05,PHARM_CLASSES__CalciumChannelAntagonistsMoACalciumChannelBlockerEPC=0.012364524500076324,PHARM_CLASSES__CalciumChannelAntagonistsMoACalciumChannelBlockerEPCCytochromeP4503A4InhibitorsMoACytochromeP4503AInhibitorsMoAPGlycoproteinInhibitorsMoA=0.0015264845061822622,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredient=0.014730575484658831,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC=0.0046557777438558995,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.0016028087314913753,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientAngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoADecreasedBloodPressurePE=0.003358265913600977,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4502D6InhibitorsMoACytochromeP4502C8InhibitorsMoACytochromeP4502C19InhibitorsMoA=0.00061059380247290495,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientHMGCoAReductaseInhibitorEPCHydroxymethylglutarylCoAReductaseInhibitorsMoA=0.00099221492901847039,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC=0.00030529690123645248,PHARM_CLASSES__CalciumsensingReceptorAgonistEPCIncreasedCalciumsensingReceptorSensitivityMoA=0.00022897267592733933,PHARM_CLASSES__CannabinoidEPCCannabinoidsChemicalIngredient=0.00061059380247290495,PHARM_CLASSES__CarbonicAnhydraseInhibitorEPCCarbonicAnhydraseInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__CarbonicAnhydraseInhibitorEPCCarbonicAnhydraseInhibitorsMoAAdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPC=0.00030529690123645248,PHARM_CLASSES__CarbonicAnhydraseInhibitorEPCCarbonicAnhydraseInhibitorsMoASulfonamidesChemicalIngredient=0.00053426957716379178,PHARM_CLASSES__CardiacGlycosideEPCCardiacGlycosidesChemicalIngredient=0.0017554571821096017,PHARM_CLASSES__CarnitineChemicalIngredientCarnitineAnalogEPC=0.00030529690123645248,PHARM_CLASSES__CatecholOMethyltransferaseInhibitorsMoACatecholOMethyltransferaseInhibitorEPC=0.00038162112654556554,PHARM_CLASSES__CellDeathInducerEPCIncreasedCellularDeathPE=0.00015264845061822624,PHARM_CLASSES__CentralNervousSystemStimulantEPCCentralNervousSystemStimulationPE=0.011067012669821401,PHARM_CLASSES__CentralNervousSystemStimulantEPCCentralNervousSystemStimulationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPE=0.0035109143642192034,PHARM_CLASSES__CentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredientFullOpioidAgonistsMoAOpioidAgonistEPC=7.6324225309113119e05,PHARM_CLASSES__CentralNervousSystemStimulationPEIncreasedSympatheticActivityPESympathomimeticlikeAgentEPC=0.0025950236605098457,PHARM_CLASSES__CentrallymediatedMuscleRelaxationPE=0.00053426957716379178,PHARM_CLASSES__CentrallymediatedMuscleRelaxationPEMuscleRelaxantEPC=0.0057243168981834834,PHARM_CLASSES__CephalosporinAntibacterialEPCCephalosporinsChemicalIngredient=0.01129598534574874,PHARM_CLASSES__ChlorideChannelActivatorEPCChlorideChannelActivatorsMoA=0.00015264845061822624,PHARM_CLASSES__CholecalciferolChemicalIngredientVitaminD3AnalogEPC=0.0018317814074187146,PHARM_CLASSES__CholinergicAgonistsMoACholinergicMuscarinicAgonistsMoACholinergicReceptorAgonistEPC=0.00030529690123645248,PHARM_CLASSES__CholinergicMuscarinicAgonistEPCCholinergicMuscarinicAgonistsMoA=0.0010685391543275836,PHARM_CLASSES__CholinergicMuscarinicAgonistsMoACholinergicReceptorAgonistEPC=0.00030529690123645248,PHARM_CLASSES__CholinergicMuscarinicAntagonistEPCCholinergicMuscarinicAntagonistsMoA=0.0052663715463288048,PHARM_CLASSES__CholinesteraseInhibitorEPCCholinesteraseInhibitorsMoA=0.010456418867348497,PHARM_CLASSES__CollagenasesChemicalIngredientCollagenspecificEnzymeEPC=0.00015264845061822624,PHARM_CLASSES__CompetitiveOpioidAntagonistsMoAPartialOpioidAgonistAntagonistEPCPartialOpioidAgonistsMoAOpioidAntagonistEPCOpioidAntagonistsMoA=7.6324225309113119e05,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.018546786750114487,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoAAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoAAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoACorticosteroidHormoneReceptorAgonistsMoA=0.0012975118302549229,PHARM_CLASSES__CyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC=0.0041215081666921083,PHARM_CLASSES__CyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPEDecreasedPlateletAggregationPEPlateletAggregationInhibitorEPC=7.6324225309113119e05,PHARM_CLASSES__CyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCProstaglandinE1AnalogEPCProstaglandinsESyntheticChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC=0.022973591818043048,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCCyclooxygenaseInhibitorEPC=0.0011448633796366967,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCHistamineH2ReceptorAntagonistsMoAHistamine2ReceptorAntagonistEPC=7.6324225309113119e05,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCProtonPumpInhibitorEPCProtonPumpInhibitorsMoACytochromeP4502C19InhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__CytochromeP45017A1InhibitorsMoACytochromeP45017A1InhibitorEPCCytochromeP4502D6InhibitorsMoACytochromeP4502C8InhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__CytochromeP4503A4InhibitorsMoAPGlycoproteinInteractionsMoA=0.00045794535185467866,PHARM_CLASSES__CytomegalovirusNucleosideAnalogDNAPolymeraseInhibitorEPCDNAPolymeraseInhibitorsMoANucleosideAnalogChemicalIngredientNucleosideAnalogAntiviralEPC=0.00038162112654556554,PHARM_CLASSES__DNAPolymeraseInhibitorsMoAHerpesSimplexVirusNucleosideAnalogDNAPolymeraseInhibitorEPCHerpesZosterVirusNucleosideAnalogDNAPolymeraseInhibitorEPCHerpesvirusNucleosideAnalogDNAPolymeraseInhibitorEPCNucleosideAnalogChemicalIngredient=0.0049610746450923521,PHARM_CLASSES__DNAPolymeraseInhibitorsMoAHerpesvirusNucleosideAnalogDNAPolymeraseInhibitorEPCNucleosideAnalogChemicalIngredient=0.0012975118302549229,PHARM_CLASSES__DecreasedCellWallIntegrityPE=0.00022897267592733933,PHARM_CLASSES__DecreasedCellWallSynthesis&amp;RepairPEAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__DecreasedCellWallSynthesis&amp;RepairPEPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__DecreasedCentralNervousSystemDisorganizedElectricalActivityPE=0.0094642039383300257,PHARM_CLASSES__DecreasedCentralNervousSystemDisorganizedElectricalActivityPECytochromeP4503A4InducersMoACytochromeP4502C19InhibitorsMoA=0.0074797740802930853,PHARM_CLASSES__DecreasedCentralNervousSystemDisorganizedElectricalActivityPEMoodStabilizerEPCCytochromeP4503A4InducersMoACytochromeP4501A2InducersMoACytochromeP4502B6InducersMoACytochromeP4502C9InducersMoACytochromeP4502C19InducersMoA=0.0015264845061822622,PHARM_CLASSES__DecreasedCholesterolAbsorptionPEDietaryCholesterolAbsorptionInhibitorEPC=0.00068691802778201802,PHARM_CLASSES__DecreasedCholesterolAbsorptionPEDietaryCholesterolAbsorptionInhibitorEPCHMGCoAReductaseInhibitorEPCHydroxymethylglutarylCoAReductaseInhibitorsMoA=0.0011448633796366967,PHARM_CLASSES__DecreasedDNAReplicationPEDecreasedProteinSynthesisPEDecreasedRNAReplicationPEProteinSynthesisInhibitorsMoA=0.001373836055564036,PHARM_CLASSES__DecreasedHistamineReleasePEHistamineH1ReceptorAntagonistsMoAHistamine1ReceptorInhibitorEPC=0.00053426957716379178,PHARM_CLASSES__DecreasedHistamineReleasePEMastCellStabilizerEPC=0.00045794535185467866,PHARM_CLASSES__DecreasedImmunologicActivityPEmTORInhibitorImmunosuppressantEPCmTORInhibitorsMoAKinaseInhibitorEPCProteinKinaseInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__DecreasedMitosisPE=0.00015264845061822624,PHARM_CLASSES__DecreasedMitosisPEMicrotubuleInhibitionPETubulinInhibitingAgentEPC=0.00091589070370935732,PHARM_CLASSES__DecreasedPlateletAggregationPEP2Y12PlateletInhibitorEPCP2Y12ReceptorAntagonistsMoACytochromeP4502C8InhibitorsMoA=0.0022897267592733935,PHARM_CLASSES__DecreasedPlateletAggregationPEPlateletAggregationInhibitorEPC=0.00045794535185467866,PHARM_CLASSES__DecreasedProteinSynthesisPEDecreasedSebaceousGlandActivityPE=0.00015264845061822624,PHARM_CLASSES__DecreasedRenalK+ExcretionPEIncreasedDiuresisPEPotassiumsparingDiureticEPC=0.00022897267592733933,PHARM_CLASSES__DecreasedRenalK+ExcretionPEIncreasedDiuresisPEPotassiumsparingDiureticEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.00045794535185467866,PHARM_CLASSES__DecreasedRespiratorySecretionViscosityPEDeoxyribonucleaseIChemicalIngredientRecombinantHumanDeoxyribonuclease1EPCRecombinantProteinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__DecreasedSebaceousGlandActivityPELincosamideAntibacterialEPCLincosamidesChemicalIngredient=0.0022134025339642801,PHARM_CLASSES__DecreasedSebaceousGlandActivityPELincosamideAntibacterialEPCLincosamidesChemicalIngredientRetinoidEPCRetinoidsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__DecreasedSebaceousGlandActivityPEMacrolideEPCMacrolideAntimicrobialEPCMacrolidesChemicalIngredient=0.0020607540833460542,PHARM_CLASSES__DecreasedStriatedMuscleContractionPEDecreasedStriatedMuscleTonePESkeletalMuscleRelaxantEPC=0.00045794535185467866,PHARM_CLASSES__DecreasedTracheobronchialStretchReceptorActivityPENonnarcoticAntitussiveEPC=0.0021370783086551671,PHARM_CLASSES__DemulcentEPCDemulcentActivityMoA=7.6324225309113119e05,PHARM_CLASSES__DepigmentingActivityPEMelaninSynthesisInhibitorEPCMelaninSynthesisInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__DihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__DihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoAPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__DihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientCalciumChannelAntagonistsMoADecreasedBloodPressurePE=0.00022897267592733933,PHARM_CLASSES__DipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPC=0.00030529690123645248,PHARM_CLASSES__DipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPCBiguanideEPCBiguanidesChemicalIngredient=0.00022897267592733933,PHARM_CLASSES__DipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPCPeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__DopamineAgonistsMoANonergotDopamineAgonistEPC=0.010151121966112044,PHARM_CLASSES__DopamineD2AntagonistsMoADopamine2ReceptorAntagonistEPC=0.00083956647840024426,PHARM_CLASSES__EpoxideHydrolaseInhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__ErgocalciferolsChemicalIngredientProvitaminD2CompoundEPC=0.00053426957716379178,PHARM_CLASSES__ErgocalciferolsChemicalIngredientVitaminD2AnalogEPC=7.6324225309113119e05,PHARM_CLASSES__ErgolinesChemicalIngredientErgotDerivativeEPC=0.00038162112654556554,PHARM_CLASSES__ErgotamineDerivativeEPCErgotaminesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__ErythropoiesisstimulatingAgentEPCErythropoietinChemicalIngredientIncreasedErythroidCellProductionPE=0.00030529690123645248,PHARM_CLASSES__EsterLocalAnestheticEPCEstersChemicalIngredientLocalAnesthesiaPE=7.6324225309113119e05,PHARM_CLASSES__EstradiolCongenersChemicalIngredientEstrogenEPCEstrogenReceptorAgonistsMoA=0.0022897267592733935,PHARM_CLASSES__EstradiolCongenersChemicalIngredientEstrogenEPCEstrogenReceptorAgonistsMoAInhibitOvumFertilizationPEProgesteroneCongenersChemicalIngredientProgesteroneCongenersChemicalIngredientProgestinEPCProgestincontainingIntrauterineDeviceEPC=7.6324225309113119e05,PHARM_CLASSES__EstradiolCongenersChemicalIngredientEstrogenEPCEstrogenReceptorAgonistsMoAProgesteroneCongenersChemicalIngredientProgestinEPC=0.00038162112654556554,PHARM_CLASSES__EstrogenAgonistAntagonistEPCSelectiveEstrogenReceptorModulatorsMoA=0.0032056174629827507,PHARM_CLASSES__EstrogenEPCEstrogenReceptorAgonistsMoAEstrogensConjugatedUSPChemicalIngredientAndrogenEPCAndrogenReceptorAgonistsMoAAndrostanesChemicalIngredient=0.00015264845061822624,PHARM_CLASSES__EstrogenEPCEstrogenReceptorAgonistsMoAProgesteroneCongenersChemicalIngredientProgestinEPC=0.00038162112654556554,PHARM_CLASSES__FactorVIIIActivatorEPCIncreasedCoagulationFactorVIIIActivityPEIncreasedCoagulationFactorVIIIConcentrationPEVasopressinAnalogEPCVasopressinsChemicalIngredient=0.0010685391543275836,PHARM_CLASSES__FactorXaInhibitorEPCFactorXaInhibitorsMoA=0.00083956647840024426,PHARM_CLASSES__FactorXaInhibitorsMoAFactorXaInhibitorEPC=0.00061059380247290495,PHARM_CLASSES__FattyAcidsOmega3ChemicalIngredientOmega3FattyAcidEPC=0.00038162112654556554,PHARM_CLASSES__FattyAcidsOmega3ChemicalIngredientOmega3FattyAcidEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCVitaminDChemicalIngredientVitaminDEPCCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__FattyAcidsOmega3ChemicalIngredientOmega3FattyAcidEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCVitaminDChemicalIngredientVitaminDEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=7.6324225309113119e05,PHARM_CLASSES__FolateAnalogMetabolicInhibitorEPCFolicAcidMetabolismInhibitorsMoA=0.00099221492901847039,PHARM_CLASSES__FolateAnalogEPCFolicAcidChemicalIngredientAnalogsDerivativesChemicalIngredient=0.00022897267592733933,PHARM_CLASSES__FolateAnalogEPCFolicAcidChemicalIngredientAnalogsDerivativesChemicalIngredientMethylatingActivityMoAMethylatingAgentEPCAcidifyingActivityMoACalculiDissolutionAgentEPCAnticoagulantEPCDecreasedCoagulationFactorActivityPECalciumChelatingActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=7.6324225309113119e05,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPC=0.026484506182262248,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCBarbiturateEPCBarbituratesChemicalIngredientCentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredientCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPE=7.6324225309113119e05,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPE=7.6324225309113119e05,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCOpioidAntagonistEPCOpioidAntagonistsMoA=0.00045794535185467866,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCPhenothiazineEPCPhenothiazinesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__GABAAAgonistsMoAGABABAgonistsMoAgammaAminobutyricAcidergicAgonistEPC=0.00076324225309113108,PHARM_CLASSES__GlinideEPCPotassiumChannelAntagonistsMoA=0.0019844298580369408,PHARM_CLASSES__GlycopeptideAntibacterialEPCGlycopeptidesChemicalIngredient=0.002823996336437185,PHARM_CLASSES__GlycosaminoglycanEPCGlycosaminoglycansChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__GranulocyteColonyStimulatingFactorChemicalIngredientGranulocyteMacrophageColonyStimulatingFactorChemicalIngredientIncreasedMyeloidCellProductionPELeukocyteGrowthFactorEPC=0.00015264845061822624,PHARM_CLASSES__HIVIntegraseInhibitorsMoAHumanImmunodeficiencyVirusIntegraseStrandTransferInhibitorEPCMultidrugandToxinExtrusionTransporter1InhibitorsMoAOrganicCationTransporter2InhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503AInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503AInhibitorsMoACytochromeP4502D6InhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503AInhibitorsMoACytochromeP4502D6InhibitorsMoACytochromeP4503AInhibitorsMoACytochromeP4503AInhibitorEPCPGlycoproteinInhibitorsMoACytochromeP4502D6InhibitorsMoAOrganicAnionTransportingPolypeptide1B1InhibitorsMoAOrganicAnionTransportingPolypeptide1B3InhibitorsMoABreastCancerResistanceProteinInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503A4InhibitorsMoACytochromeP4503A4InducersMoAPGlycoproteinInducersMoA=7.6324225309113119e05,PHARM_CLASSES__HMGCoAReductaseInhibitorEPCHydroxymethylglutarylCoAReductaseInhibitorsMoA=0.02694245153411693,PHARM_CLASSES__HeparinLowMolecularWeightChemicalIngredientLowMolecularWeightHeparinEPC=0.0014501602808731492,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=0.00083956647840024426,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=0.00068691802778201802,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=0.0010685391543275836,PHARM_CLASSES__HistamineH1ReceptorAntagonistsMoAHistamine1ReceptorAntagonistEPC=0.0030529690123645243,PHARM_CLASSES__HistamineH1ReceptorAntagonistsMoAHistamine1ReceptorAntagonistEPCAdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPC=7.6324225309113119e05,PHARM_CLASSES__HistamineH2ReceptorAntagonistsMoAHistamine2ReceptorAntagonistEPC=0.0048084261944741262,PHARM_CLASSES__Histamine1ReceptorAntagonistEPC=0.00015264845061822624,PHARM_CLASSES__HumanImmunodeficiencyVirus1NonNucleosideAnalogReverseTranscriptaseInhibitorEPCNonNucleosideAnalogChemicalIngredientNonNucleosideReverseTranscriptaseInhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__HumanImmunodeficiencyVirus1NonNucleosideAnalogReverseTranscriptaseInhibitorEPCNonNucleosideAnalogChemicalIngredientNonNucleosideReverseTranscriptaseInhibitorsMoACytochromeP4503AInducersMoACytochromeP4502B6InducersMoA=0.00053426957716379178,PHARM_CLASSES__HumanImmunodeficiencyVirus1NonNucleosideAnalogReverseTranscriptaseInhibitorEPCNonNucleosideReverseTranscriptaseInhibitorsMoANonNucleosideAnalogChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__HumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=0.00068691802778201802,PHARM_CLASSES__HumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHIVIntegraseInhibitorsMoAHumanImmunodeficiencyVirusIntegraseStrandTransferInhibitorEPCMultidrugandToxinExtrusionTransporter1InhibitorsMoAOrganicCationTransporter2InhibitorsMoAHepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__HumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA=0.00038162112654556554,PHARM_CLASSES__HumanPlateletderivedGrowthFactorEPCPlateletDerivedGrowthFactorChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__HyperpolarizationactivatedCyclicNucleotidegatedChannelBlockerEPCHyperpolarizationactivatedCyclicNucleotidegatedChannelAntagonistsMoA=0.00030529690123645248,PHARM_CLASSES__IncreasedCytokineActivityPEIncreasedCytokineProductionPEInterferonInducersMoA=0.00061059380247290495,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient=0.0056479926728743701,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPC=0.00076324225309113108,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC=0.0029766447870554114,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoA=0.00091589070370935732,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientDecreasedRenalK+ExcretionPEIncreasedDiuresisPEPotassiumsparingDiureticEPC=7.6324225309113119e05,PHARM_CLASSES__IncreasedDiuresisPEThiazidelikeDiureticEPC=0.00083956647840024426,PHARM_CLASSES__IncreasedDiuresisatLoopofHenlePELoopDiureticEPC=0.0037398870401465427,PHARM_CLASSES__InfluenzaAM2ProteinInhibitorEPCM2ProteinInhibitorsMoA=0.00076324225309113108,PHARM_CLASSES__InhibitOvumFertilizationPEProgesteroneCongenersChemicalIngredientProgesteroneCongenersChemicalIngredientProgestinEPCProgestincontainingIntrauterineDeviceEPCEstrogenEPCEstrogenReceptorAgonistsMoA=0.00015264845061822624,PHARM_CLASSES__Interleukin12AntagonistEPCInterleukin12AntagonistsMoAInterleukin23AntagonistEPCInterleukin23AntagonistsMoA=0.00015264845061822624,PHARM_CLASSES__IronChemicalIngredientParenteralIronReplacementEPCPhosphateBinderEPCPhosphateChelatingActivityMoA=7.6324225309113119e05,PHARM_CLASSES__KinaseInhibitorEPCProteinKinaseInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__LeukotrieneReceptorAntagonistEPCLeukotrieneReceptorAntagonistsMoA=0.0064112349259655014,PHARM_CLASSES__LeukotrieneReceptorAntagonistEPCLeukotrieneReceptorAntagonistsMoACytochromeP4502C9InhibitorsMoA=0.00053426957716379178,PHARM_CLASSES__LocalAnesthesiaPELocalAnestheticEPC=0.00022897267592733933,PHARM_CLASSES__LymphocyteFunctionAssociatedAntigen1AntagonistEPCLymphocyteFunctionAssociatedAntigen1AntagonistsMoA=7.6324225309113119e05,PHARM_CLASSES__MacrolideAntimicrobialEPCMacrolidesChemicalIngredient=0.0017554571821096017,PHARM_CLASSES__MacrolideAntimicrobialEPCMacrolidesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4503AInhibitorsMoAPGlycoproteinInhibitorsMoA=0.00099221492901847039,PHARM_CLASSES__MacrolidesChemicalIngredientMacrolideAntibacterialEPC=7.6324225309113119e05,PHARM_CLASSES__MelatoninReceptorAgonistEPCMelatoninReceptorAgonistsMoA=0.00015264845061822624,PHARM_CLASSES__MethylxanthineEPCXanthinesChemicalIngredient=0.0016791329568004885,PHARM_CLASSES__MonoamineOxidaseInhibitorEPCMonoamineOxidaseInhibitorsMoA=0.00038162112654556554,PHARM_CLASSES__MonoamineOxidaseInhibitorEPCMonoamineOxidaseInhibitorsMoAMonoamineOxidaseTypeBInhibitorEPCMonoamineOxidaseBInhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__MoodStabilizerEPC=0.0014501602808731492,PHARM_CLASSES__NMDAReceptorAntagonistsMoANmethylDaspartateReceptorAntagonistEPC=0.0029003205617462984,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredient=0.0012975118302549229,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC=7.6324225309113119e05,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPECopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE=7.6324225309113119e05,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE=7.6324225309113119e05,PHARM_CLASSES__NitrateVasodilatorEPCNitratesChemicalIngredientVasodilationPE=0.0052663715463288048,PHARM_CLASSES__NitrofuranAntibacterialEPCNitrofuransChemicalIngredient=0.00099221492901847039,PHARM_CLASSES__NitrofuranAntibacterialEPCNitrofuransChemicalIngredientNitrofuranAntibacterialEPCNitrofuransChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__NitroimidazoleAntimicrobialEPCNitroimidazolesChemicalIngredient=0.0051137230957105789,PHARM_CLASSES__NoncompetitiveAMPAGlutamateReceptorAntagonistEPCAMPAReceptorAntagonistsMoA=0.00030529690123645248,PHARM_CLASSES__NorepinephrineUptakeInhibitorsMoASerotoninandNorepinephrineReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoA=0.014577927034040605,PHARM_CLASSES__NorepinephrineUptakeInhibitorsMoASerotoninandNorepinephrineReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoACytochromeP4502D6InhibitorsMoA=0.00091589070370935732,PHARM_CLASSES__NucleicAcidSynthesisInhibitorsMoANucleosideAnalogChemicalIngredientPurineAntimetaboliteEPCPurinesChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__NucleicAcidSynthesisInhibitorsMoANucleosideMetabolicInhibitorEPC=0.0010685391543275836,PHARM_CLASSES__NucleosideAnalogChemicalIngredientNucleosideAnalogAntiviralEPC=0.0011448633796366967,PHARM_CLASSES__NucleosideAnalogChemicalIngredientNucleosideAnalogAntiviralEPCNucleosideMetabolicInhibitorEPCNucleicAcidSynthesisInhibitorsMoA=0.00022897267592733933,PHARM_CLASSES__OpioidAgonistEPCOpioidAgonistsMoA=0.0089299343611662346,PHARM_CLASSES__OpioidAgonistEPCOpioidAgonistsMoACyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC=0.00061059380247290495,PHARM_CLASSES__OpioidAgonistEPCOpioidAgonistsMoAHistamineH1ReceptorAntagonistsMoAHistamine1ReceptorAntagonistEPC=0.00061059380247290495,PHARM_CLASSES__OpioidAntagonistEPCOpioidAntagonistsMoA=0.00083956647840024426,PHARM_CLASSES__OpioidAntagonistEPCOpioidAntagonistsMoAAminoketoneEPCDopamineUptakeInhibitorsMoAIncreasedDopamineActivityPEIncreasedNorepinephrineActivityPENorepinephrineUptakeInhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__OpioidAntagonistEPCOpioidAntagonistsMoAQuaternaryAmmoniumCompoundsChemicalIngredient=0.00022897267592733933,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPE=0.00038162112654556554,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=7.6324225309113119e05,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=0.00022897267592733933,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=0.00015264845061822624,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=0.00015264845061822624,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=7.6324225309113119e05,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=0.00015264845061822624,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoACalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=7.6324225309113119e05,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE=0.00015264845061822624,PHARM_CLASSES__OsmoticLaxativeEPCOsmoticActivityMoAAcidifyingActivityMoAStimulationLargeIntestineFluidElectrolyteSecretionPE=0.00099221492901847039,PHARM_CLASSES__OxaboroleAntifungalEPCProteinSynthesisInhibitorsMoABoronCompoundsChemicalIngredient=0.00015264845061822624,PHARM_CLASSES__OxazolidinoneAntibacterialEPCOxazolidinonesChemicalIngredient=0.00091589070370935732,PHARM_CLASSES__PCSK9InhibitorEPCPCSK9InhibitorsMoAAntibodiesMonoclonalChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__PartialOpioidAgonistEPCPartialOpioidAgonistsMoA=0.00083956647840024426,PHARM_CLASSES__PartialOpioidAgonistEPCPartialOpioidAgonistsMoAOpioidAntagonistEPCOpioidAntagonistsMoA=0.00076324225309113108,PHARM_CLASSES__PediculicideEPC=7.6324225309113119e05,PHARM_CLASSES__PenicillinclassAntibacterialEPCPenicillinsChemicalIngredient=0.006334910700656388,PHARM_CLASSES__PenicillinclassAntibacterialEPCPenicillinsChemicalIngredientbetaLactamaseInhibitorEPCbetaLactamaseInhibitorsMoA=0.0026713478858189591,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPC=0.0029766447870554114,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredient=0.0030529690123645243,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredientBiguanideEPCBiguanidesChemicalIngredient=0.00068691802778201802,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredientSulfonylureaEPCSulfonylureaCompoundsChemicalIngredient=0.00030529690123645248,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatoractivatedReceptoralphaAgonistsMoAPPARalphaChemicalIngredient=0.0029766447870554114,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredient=0.0048847504197832396,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientFullOpioidAgonistsMoAOpioidAgonistEPC=0.00022897267592733933,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientUncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoA=7.6324225309113119e05,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientalpha1AdrenergicAgonistEPCAdrenergicalpha1AgonistsMoA=7.6324225309113119e05,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientalpha1AdrenergicAgonistEPCAdrenergicalpha1AgonistsMoAFullOpioidAgonistsMoAOpioidAgonistEPC=0.00030529690123645248,PHARM_CLASSES__PhosphateBinderEPCPhosphateChelatingActivityMoA=0.00015264845061822624,PHARM_CLASSES__Phosphodiesterase3InhibitorEPCPhosphodiesterase3InhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__Phosphodiesterase4InhibitorEPCPhosphodiesterase4InhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__Phosphodiesterase5InhibitorEPCPhosphodiesterase5InhibitorsMoA=0.00099221492901847039,PHARM_CLASSES__PolyeneAntifungalEPCPolyenesChemicalIngredient=0.0025186994352007328,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoAAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDecreasedCellWallSynthesis&amp;RepairPE=7.6324225309113119e05,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDecreasedCellWallSynthesis&amp;RepairPEAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoA=0.00022897267592733933,PHARM_CLASSES__PotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=0.0058769653488017094,PHARM_CLASSES__ProgesteroneCongenersChemicalIngredientProgestinEPC=0.0021370783086551671,PHARM_CLASSES__ProgesteroneCongenersChemicalIngredientProgestinEPCEstrogenEPCEstrogenReceptorAgonistsMoA=0.00076324225309113108,PHARM_CLASSES__ProgesteroneChemicalIngredientProgesteroneEPC=0.00091589070370935732,PHARM_CLASSES__ProstaglandinAnalogEPCProstaglandinReceptorAgonistsMoAIncreasedProstaglandinActivityPEProstaglandinsChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__ProstaglandinAnalogEPCProstaglandinsChemicalIngredient=0.00045794535185467866,PHARM_CLASSES__ProstaglandinE1AnalogEPCProstaglandinsESyntheticChemicalIngredient=0.00061059380247290495,PHARM_CLASSES__ProteaseactivatedReceptor1AntagonistEPCProteaseactivatedReceptor1AntagonistsMoA=0.00015264845061822624,PHARM_CLASSES__ProteasomeInhibitorEPCProteasomeInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__ProtonPumpInhibitorEPCProtonPumpInhibitorsMoA=0.0038162112654556556,PHARM_CLASSES__ProtonPumpInhibitorEPCProtonPumpInhibitorsMoACytochromeP4502C19InhibitorsMoA=0.0083193405586933292,PHARM_CLASSES__ProtonPumpInhibitorEPCProtonPumpInhibitorsMoAInhibitionGastricAcidSecretionPE=0.0029003205617462984,PHARM_CLASSES__PyrethrinsChemicalIngredientPyrethroidEPC=0.00015264845061822624,PHARM_CLASSES__PyrimidineSynthesisInhibitorEPCDihydroorotateDehydrogenaseInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__QuinoloneAntimicrobialEPCQuinolonesChemicalIngredient=0.0051137230957105789,PHARM_CLASSES__QuinoloneAntimicrobialEPCQuinolonesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=7.6324225309113119e05,PHARM_CLASSES__RANKLigandInhibitorEPCRANKLigandBlockingActivityMoA=7.6324225309113119e05,PHARM_CLASSES__RNASynthetaseInhibitorAntibacterialEPCRNASynthetaseInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__ReninInhibitorEPCReninInhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__RetinoidEPCRetinoidsChemicalIngredient=0.0045031292932376735,PHARM_CLASSES__RifamycinAntibacterialEPCRifamycinsChemicalIngredient=0.0015264845061822622,PHARM_CLASSES__Serotonin1bReceptorAgonistsMoASerotonin1dReceptorAgonistsMoASerotonin1bandSerotonin1dReceptorAgonistEPC=0.0083956647840024417,PHARM_CLASSES__Serotonin1bReceptorAgonistsMoASerotonin1dReceptorAgonistsMoASerotonin1bandSerotonin1dReceptorAgonistEPCCyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC=7.6324225309113119e05,PHARM_CLASSES__Serotonin3ReceptorAntagonistsMoASerotonin3ReceptorAntagonistEPC=0.0055716684475652575,PHARM_CLASSES__SerotoninReuptakeInhibitorEPC=0.0016028087314913753,PHARM_CLASSES__SerotoninReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoA=0.029003205617462984,PHARM_CLASSES__Serotonin2cReceptorAgonistEPCSerotonin2cReceptorAgonistsMoACytochromeP4502D6InhibitorsMoA=0.00015264845061822624,PHARM_CLASSES__SkinTestAntigenEPCTuberculosisSkinTestEPCCellmediatedImmunityPEAntigensBacterialChemicalIngredient=0.00015264845061822624,PHARM_CLASSES__SodiumGlucoseCotransporter2InhibitorEPCSodiumGlucoseTransporter2InhibitorsMoAPGlycoproteinInhibitorsMoA=0.00030529690123645248,PHARM_CLASSES__SodiumGlucoseCotransporter2InhibitorEPCSodiumGlucoseTransporter2InhibitorsMoAPGlycoproteinInhibitorsMoABiguanideEPCBiguanidesChemicalIngredient=0.00038162112654556554,PHARM_CLASSES__SomatostatinAnalogEPCSomatostatinReceptorAgonistsMoA=0.00030529690123645248,PHARM_CLASSES__SulfonamideAntibacterialEPCSulfonamidesChemicalIngredient=0.0019081056327278278,PHARM_CLASSES__SulfonamideAntibacterialEPCSulfonamidesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00030529690123645248,PHARM_CLASSES__SulfonamideAntimicrobialEPCSulfonamidesChemicalIngredientCytochromeP4502C9InhibitorsMoADihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoA=0.0022134025339642801,PHARM_CLASSES__SulfoneEPCSulfonesChemicalIngredient=0.001373836055564036,PHARM_CLASSES__SulfonylureaEPCSulfonylureaCompoundsChemicalIngredient=0.0084719890093115559,PHARM_CLASSES__SulfonylureaEPCSulfonylureaCompoundsChemicalIngredientBiguanideEPCBiguanidesChemicalIngredient=0.0022897267592733935,PHARM_CLASSES__TetracyclineclassAntimicrobialEPCTetracyclinesChemicalIngredient=0.0010685391543275836,PHARM_CLASSES__TetracyclineclassDrugEPCTetracyclinesChemicalIngredient=0.0023660509845825064,PHARM_CLASSES__ThyroidHormoneSynthesisInhibitorEPCThyroidHormoneSynthesisInhibitorsMoA=0.00091589070370935732,PHARM_CLASSES__TricyclicAntidepressantEPC=0.0065638833765837273,PHARM_CLASSES__TumorNecrosisFactorBlockerEPCTumorNecrosisFactorReceptorBlockingActivityMoA=0.00053426957716379178,PHARM_CLASSES__TypicalAntipsychoticEPC=0.0017554571821096017,PHARM_CLASSES__UncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoAAdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPC=0.00015264845061822624,PHARM_CLASSES__UncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoAAntiarrhythmicEPCCytochromeP4502D6InhibitorEPCCytochromeP4502D6InhibitorsMoA=7.6324225309113119e05,PHARM_CLASSES__UncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoAPhenothiazineEPCPhenothiazinesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPC=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminB12ChemicalIngredientVitaminB12EPCVitaminDChemicalIngredientVitaminDEPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCIronChemicalIngredientParenteralIronReplacementEPCPhosphateBinderEPCPhosphateChelatingActivityMoA=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB12ChemicalIngredientVitaminB12EPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient=0.00015264845061822624,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC=0.00030529690123645248,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE=0.00015264845061822624,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__VitaminB12ChemicalIngredientVitaminB12EPC=0.00030529690123645248,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientVitaminDChemicalIngredientVitaminDEPCVitaminB12ChemicalIngredientVitaminB12EPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPC=7.6324225309113119e05,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC=0.00022897267592733933,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE=7.6324225309113119e05,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCVitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPC=7.6324225309113119e05,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDAnalogEPC=0.00068691802778201802,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDAnalogEPCCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA=0.00061059380247290495,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientAntidoteEPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB12ChemicalIngredientVitaminB12EPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient=7.6324225309113119e05,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=0.00015264845061822624,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPE=0.00015264845061822624,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCalciumChemicalIngredientCalciumEPCPhosphateBinderEPCPhosphateChelatingActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA=7.6324225309113119e05,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE=7.6324225309113119e05,PHARM_CLASSES__VitaminKAntagonistEPCVitaminKInhibitorsMoA=0.0058006411234925968,PHARM_CLASSES__XanthineOxidaseInhibitorEPCXanthineOxidaseInhibitorsMoA=0.002823996336437185,PHARM_CLASSES__alphaGlucosidaseInhibitorsMoAalphaGlucosidaseInhibitorEPC=0.0022134025339642801,PHARM_CLASSES__gammaAminobutyricAcidAReceptorAgonistEPCGABAAAgonistsMoACentralNervousSystemDepressionPE=0.00030529690123645248,PHARM_CLASSES__gammaAminobutyricAcidergicAgonistEPCGABAAAgonistsMoAPyridinesChemicalIngredientCentralNervousSystemDepressionPE=0.0020607540833460542,PHARM_CLASSES__lThyroxineEPCThyroxineChemicalIngredient=0.0044268050679285601,PHARM_CLASSES__lThyroxineEPCThyroxineChemicalIngredientlTriiodothyronineEPCTriiodothyronineChemicalIngredient=0.00022897267592733933,PHARM_CLASSES__lTriiodothyronineEPCTriiodothyronineChemicalIngredient=0.00061059380247290495,PHARM_CLASSES__muOpioidReceptorAgonistEPCOpioidmuReceptorAgonistsMoA=0.00015264845061822624,priceType=0.9402381315829644 ):
    return list(STARTMARKETINGDATE,DOSAGEFORMNAME__AEROSOLFOAM,DOSAGEFORMNAME__AEROSOLMETERED,DOSAGEFORMNAME__AEROSOLPOWDER,DOSAGEFORMNAME__AEROSOLSPRAY,DOSAGEFORMNAME__CAPSULE,DOSAGEFORMNAME__CAPSULECOATEDPELLETS,DOSAGEFORMNAME__CAPSULECOATEDEXTENDEDRELEASE,DOSAGEFORMNAME__CAPSULEDELAYEDRELEASE,DOSAGEFORMNAME__CAPSULEDELAYEDRELEASEPELLETS,DOSAGEFORMNAME__CAPSULEEXTENDEDRELEASE,DOSAGEFORMNAME__CAPSULEGELATINCOATED,DOSAGEFORMNAME__CAPSULELIQUIDFILLED,DOSAGEFORMNAME__CONCENTRATE,DOSAGEFORMNAME__CREAM,DOSAGEFORMNAME__CREAMAUGMENTED,DOSAGEFORMNAME__ELIXIR,DOSAGEFORMNAME__EMULSION,DOSAGEFORMNAME__ENEMA,DOSAGEFORMNAME__FILM,DOSAGEFORMNAME__FILMEXTENDEDRELEASE,DOSAGEFORMNAME__FORSOLUTION,DOSAGEFORMNAME__FORSUSPENSION,DOSAGEFORMNAME__GEL,DOSAGEFORMNAME__GELDENTIFRICE,DOSAGEFORMNAME__GRANULE,DOSAGEFORMNAME__GRANULEFORSOLUTION,DOSAGEFORMNAME__GRANULEFORSUSPENSION,DOSAGEFORMNAME__GUMCHEWING,DOSAGEFORMNAME__INJECTION,DOSAGEFORMNAME__INJECTIONPOWDERFORSOLUTION,DOSAGEFORMNAME__INJECTIONPOWDERLYOPHILIZEDFORSOLUTION,DOSAGEFORMNAME__INJECTIONSOLUTION,DOSAGEFORMNAME__INJECTIONSUSPENSION,DOSAGEFORMNAME__INJECTIONSUSPENSIONEXTENDEDRELEASE,DOSAGEFORMNAME__JELLY,DOSAGEFORMNAME__KIT,DOSAGEFORMNAME__LIQUID,DOSAGEFORMNAME__LOTION,DOSAGEFORMNAME__LOZENGE,DOSAGEFORMNAME__OIL,DOSAGEFORMNAME__OINTMENT,DOSAGEFORMNAME__PASTE,DOSAGEFORMNAME__PATCH,DOSAGEFORMNAME__PATCHEXTENDEDRELEASE,DOSAGEFORMNAME__PELLET,DOSAGEFORMNAME__POWDER,DOSAGEFORMNAME__POWDERFORSOLUTION,DOSAGEFORMNAME__POWDERFORSUSPENSION,DOSAGEFORMNAME__POWDERMETERED,DOSAGEFORMNAME__RINSE,DOSAGEFORMNAME__SHAMPOO,DOSAGEFORMNAME__SHAMPOOSUSPENSION,DOSAGEFORMNAME__SOLUTION,DOSAGEFORMNAME__SOLUTIONCONCENTRATE,DOSAGEFORMNAME__SOLUTIONGELFORMINGDROPS,DOSAGEFORMNAME__SOLUTIONGELFORMINGEXTENDEDRELEASE,DOSAGEFORMNAME__SOLUTIONDROPS,DOSAGEFORMNAME__SPRAY,DOSAGEFORMNAME__SPRAYMETERED,DOSAGEFORMNAME__SUPPOSITORY,DOSAGEFORMNAME__SUSPENSION,DOSAGEFORMNAME__SUSPENSIONEXTENDEDRELEASE,DOSAGEFORMNAME__SUSPENSIONDROPS,DOSAGEFORMNAME__SWAB,DOSAGEFORMNAME__SYRUP,DOSAGEFORMNAME__TABLET,DOSAGEFORMNAME__TABLETCHEWABLE,DOSAGEFORMNAME__TABLETCOATED,DOSAGEFORMNAME__TABLETDELAYEDRELEASE,DOSAGEFORMNAME__TABLETEFFERVESCENT,DOSAGEFORMNAME__TABLETEXTENDEDRELEASE,DOSAGEFORMNAME__TABLETFILMCOATED,DOSAGEFORMNAME__TABLETFILMCOATEDEXTENDEDRELEASE,DOSAGEFORMNAME__TABLETFORSUSPENSION,DOSAGEFORMNAME__TABLETMULTILAYER,DOSAGEFORMNAME__TABLETMULTILAYEREXTENDEDRELEASE,DOSAGEFORMNAME__TABLETORALLYDISINTEGRATING,DOSAGEFORMNAME__TABLETORALLYDISINTEGRATINGDELAYEDRELEASE,DOSAGEFORMNAME__TABLETSUGARCOATED,DOSAGEFORMNAME__TINCTURE,ROUTENAME__AURICULAR(OTIC),ROUTENAME__BUCCAL,ROUTENAME__BUCCAL;SUBLINGUAL,ROUTENAME__CUTANEOUS,ROUTENAME__DENTAL,ROUTENAME__ENTERAL,ROUTENAME__INTRAARTERIAL;INTRAMUSCULAR;INTRATHECAL;INTRAVENOUS,ROUTENAME__INTRAARTERIAL;INTRAMUSCULAR;INTRAVENOUS,ROUTENAME__INTRAARTICULAR;INTRALESIONAL;INTRAMUSCULAR;INTRAVENOUS;SOFTTISSUE,ROUTENAME__INTRAARTICULAR;INTRALESIONAL;INTRAMUSCULAR;SOFTTISSUE,ROUTENAME__INTRADERMAL,ROUTENAME__INTRALESIONAL;INTRAMUSCULAR;INTRASYNOVIAL;SOFTTISSUE,ROUTENAME__INTRAMUSCULAR,ROUTENAME__INTRAMUSCULAR;INTRAVENOUS,ROUTENAME__INTRAMUSCULAR;INTRAVENOUS;SUBCUTANEOUS,ROUTENAME__INTRAMUSCULAR;SUBCUTANEOUS,ROUTENAME__INTRAOCULAR,ROUTENAME__INTRAVENOUS,ROUTENAME__INTRAVENOUS;SUBCUTANEOUS,ROUTENAME__NASAL,ROUTENAME__OPHTHALMIC,ROUTENAME__OPHTHALMIC;TOPICAL,ROUTENAME__ORAL,ROUTENAME__ORAL;ORAL,ROUTENAME__ORAL;RECTAL,ROUTENAME__ORAL;RESPIRATORY(INHALATION),ROUTENAME__ORAL;SUBLINGUAL,ROUTENAME__ORAL;TOPICAL,ROUTENAME__ORAL;TRANSMUCOSAL,ROUTENAME__PERINEURAL,ROUTENAME__RECTAL,ROUTENAME__RECTAL;TOPICAL,ROUTENAME__RESPIRATORY(INHALATION),ROUTENAME__SUBCUTANEOUS,ROUTENAME__SUBLINGUAL,ROUTENAME__TOPICAL,ROUTENAME__TRANSDERMAL,ROUTENAME__VAGINAL,PHARM_CLASSES__5LipoxygenaseInhibitorEPC5LipoxygenaseInhibitorsMoADecreasedLeukotrieneProductionPE,PHARM_CLASSES__5alphaReductaseInhibitorEPC5alphaReductaseInhibitorsMoA,PHARM_CLASSES__5alphaReductaseInhibitorEPC5alphaReductaseInhibitorsMoAAdrenergicalphaAntagonistsMoAalphaAdrenergicBlockerEPC,PHARM_CLASSES__AcetylAldehydeDehydrogenaseInhibitorsMoAAldehydeDehydrogenaseInhibitorEPC,PHARM_CLASSES__AcidifyingActivityMoACalculiDissolutionAgentEPCAnticoagulantEPCDecreasedCoagulationFactorActivityPECalciumChelatingActivityMoA,PHARM_CLASSES__AdrenergicAgonistsMoAAdrenergicReceptorAgonistEPCDecreasedHistamineReleasePEHistamineH1ReceptorAntagonistsMoAHistamine1ReceptorInhibitorEPC,PHARM_CLASSES__AdrenergicalphaAgonistsMoAAdrenergicbetaAgonistsMoAalphaAdrenergicAgonistEPCbetaAdrenergicAgonistEPCCatecholamineEPCCatecholaminesChemicalIngredient,PHARM_CLASSES__AdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPC,PHARM_CLASSES__AdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPCUncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoA,PHARM_CLASSES__AdrenergicalphaAntagonistsMoAAdrenergicbeta1AntagonistsMoAAdrenergicbeta2AntagonistsMoAalphaAdrenergicBlockerEPCbetaAdrenergicBlockerEPC,PHARM_CLASSES__AdrenergicalphaAntagonistsMoAalphaAdrenergicBlockerEPC,PHARM_CLASSES__Adrenergicalpha2AgonistsMoACentralalpha2AdrenergicAgonistEPC,PHARM_CLASSES__AdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPC,PHARM_CLASSES__AdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__Adrenergicbeta2AgonistsMoAbeta2AdrenergicAgonistEPC,PHARM_CLASSES__Adrenergicbeta2AgonistsMoAbeta2AdrenergicAgonistEPCAnticholinergicEPCCholinergicAntagonistsMoA,PHARM_CLASSES__AldosteroneAntagonistEPCAldosteroneAntagonistsMoA,PHARM_CLASSES__AldosteroneAntagonistEPCAldosteroneAntagonistsMoAIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__AlkylatingActivityMoAAlkylatingDrugEPC,PHARM_CLASSES__AllylamineChemicalIngredientAllylamineAntifungalEPC,PHARM_CLASSES__AluminumComplexEPCOrganometallicCompoundsChemicalIngredient,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientAntiarrhythmicEPCLocalAnesthesiaPE,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientAntiarrhythmicEPCLocalAnesthesiaPEAmideLocalAnestheticEPCAmidesChemicalIngredientLocalAnesthesiaPE,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientAntiarrhythmicEPCLocalAnesthesiaPECorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__AmideLocalAnestheticEPCAmidesChemicalIngredientLocalAnesthesiaPE,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__AminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDecreasedCellWallSynthesis&amp;RepairPECorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__AminoketoneEPCDopamineUptakeInhibitorsMoAIncreasedDopamineActivityPEIncreasedNorepinephrineActivityPENorepinephrineUptakeInhibitorsMoA,PHARM_CLASSES__AminosalicylateEPCAminosalicylicAcidsChemicalIngredient,PHARM_CLASSES__AmylinAgonistsMoAAmylinAnalogEPC,PHARM_CLASSES__AndrogenReceptorAntagonistsMoAAndrogenReceptorInhibitorEPC,PHARM_CLASSES__AndrogenEPCAndrogenReceptorAgonistsMoAAndrostanesChemicalIngredient,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCCalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredient,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCCalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__Angiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__Angiotensin2Type1ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCDecreasedBloodPressurePE,PHARM_CLASSES__Angiotensin2Type1ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCDecreasedBloodPressurePEIncreasedDiuresisPEThiazidelikeDiureticEPC,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoA,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoACalciumChannelAntagonistsMoACalciumChannelBlockerEPCCytochromeP4503A4InhibitorsMoACytochromeP4503AInhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoADecreasedBloodPressurePE,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoADecreasedBloodPressurePEIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__AngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoAIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__AnticoagulantEPCHeparinChemicalIngredientUnfractionatedHeparinEPC,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPE,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPECytochromeP4501A2InducersMoACytochromeP4502B6InducersMoACytochromeP4502C8InducersMoACytochromeP4502C19InducersMoACytochromeP4502D6InducersMoACytochromeP4503AInducersMoACytochromeP4502C9InducersMoA,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPEMoodStabilizerEPC,PHARM_CLASSES__AntiepilepticAgentEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPESulfonamidesChemicalIngredientCarbonicAnhydraseInhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__AntiarrhythmicEPC,PHARM_CLASSES__AntiarrhythmicEPCCardiacRhythmAlterationPEAdrenergicbetaAntagonistsMoA,PHARM_CLASSES__AntiarrhythmicEPCCytochromeP4503A4InhibitorsMoAPGlycoproteinInhibitorsMoACytochromeP4501A2InhibitorsMoACytochromeP4502C9InhibitorsMoACytochromeP4502D6InhibitorsMoA,PHARM_CLASSES__AnticholinergicEPCAntihistamineEPCCholinergicAntagonistsMoAHistamineReceptorAntagonistsMoA,PHARM_CLASSES__AnticholinergicEPCCholinergicAntagonistsMoA,PHARM_CLASSES__AnticholinergicEPCCholinergicAntagonistsMoAAdrenergicbeta2AgonistsMoAbeta2AdrenergicAgonistEPC,PHARM_CLASSES__AnticholinergicEPCCholinergicAntagonistsMoACholinergicMuscarinicAntagonistEPCCholinergicMuscarinicAntagonistsMoA,PHARM_CLASSES__AntidoteEPCAntidoteforAcetaminophenOverdoseEPCDecreasedRespiratorySecretionViscosityPEIncreasedGlutathioneConcentrationPEMucolyticEPCReductionActivityMoA,PHARM_CLASSES__AntiemeticEPCEmesisSuppressionPE,PHARM_CLASSES__AntifibrinolyticAgentEPCDecreasedFibrinolysisPE,PHARM_CLASSES__AntihelminthicEPC,PHARM_CLASSES__AntihelminthicEPCCytochromeP4501AInducersMoA,PHARM_CLASSES__AntihistamineEPCHistamineReceptorAntagonistsMoA,PHARM_CLASSES__AntihistamineEPCHistamineReceptorAntagonistsMoAVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient,PHARM_CLASSES__AntimalarialEPC,PHARM_CLASSES__AntimalarialEPCAntiprotozoalEPC,PHARM_CLASSES__AntimalarialEPCAntiprotozoalEPCAntimalarialEPCDihydrofolateReductaseInhibitorsMoA,PHARM_CLASSES__AntimetaboliteImmunosuppressantEPC,PHARM_CLASSES__AntimetaboliteEPCUreaChemicalIngredient,PHARM_CLASSES__AntimycobacterialEPC,PHARM_CLASSES__AntiparasiticEPCPediculicideEPC,PHARM_CLASSES__AntiprotozoalEPC,PHARM_CLASSES__AntirheumaticAgentEPC,PHARM_CLASSES__AntirheumaticAgentEPCAntimalarialEPC,PHARM_CLASSES__AppetiteSuppressionPEIncreasedSympatheticActivityPESympathomimeticAmineAnorecticEPC,PHARM_CLASSES__AppetiteSuppressionPEIncreasedSympatheticActivityPESympathomimeticAmineAnorecticEPCDecreasedCentralNervousSystemDisorganizedElectricalActivityPECytochromeP4503A4InducersMoACytochromeP4502C19InhibitorsMoA,PHARM_CLASSES__AromataseInhibitorEPCAromataseInhibitorsMoA,PHARM_CLASSES__AromaticAminoAcidDecarboxylationInhibitorEPCDOPADecarboxylaseInhibitorsMoAAromaticAminoAcidEPCAminoAcidsAromaticChemicalIngredient,PHARM_CLASSES__AromaticAminoAcidEPCAminoAcidsAromaticChemicalIngredient,PHARM_CLASSES__AromaticAminoAcidEPCAminoAcidsAromaticChemicalIngredientCatecholOMethyltransferaseInhibitorsMoACatecholOMethyltransferaseInhibitorEPC,PHARM_CLASSES__ArteriolarVasodilationPEArteriolarVasodilatorEPC,PHARM_CLASSES__ArteriolarVasodilationPEArteriolarVasodilatorEPCNitrateVasodilatorEPCNitratesChemicalIngredientVasodilationPE,PHARM_CLASSES__AtypicalAntipsychoticEPC,PHARM_CLASSES__AtypicalAntipsychoticEPCSerotoninReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoA,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredient,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4502C19InhibitorsMoA,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4502C19InhibitorsMoACytochromeP4503A4InhibitorsMoACytochromeP4502C9InhibitorsMoA,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4502C19InhibitorsMoACytochromeP4502C9InhibitorsMoA,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4503A5InhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__AzoleAntifungalEPCAzolesChemicalIngredientCytochromeP4503A4InhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredient,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredientCentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredient,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredientCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredient,PHARM_CLASSES__BarbiturateEPCBarbituratesChemicalIngredientCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredientFullOpioidAgonistsMoAOpioidAgonistEPC,PHARM_CLASSES__BenzodiazepineEPCBenzodiazepinesChemicalIngredient,PHARM_CLASSES__BenzodiazepineEPCBenzodiazepinesChemicalIngredientAnticholinergicEPCDecreasedParasympatheticAcetylcholineActivityPEGIMotilityAlterationPEDigestiveGISystemActivityAlterationPECholinergicAntagonistsMoA,PHARM_CLASSES__BenzodiazepineEPCBenzodiazepinesChemicalIngredientCytochromeP4502D6InhibitorsMoA,PHARM_CLASSES__BenzothiazoleEPCBenzothiazolesChemicalIngredient,PHARM_CLASSES__BiguanideEPCBiguanidesChemicalIngredient,PHARM_CLASSES__BiguanideEPCBiguanidesChemicalIngredientDipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPC,PHARM_CLASSES__BiguanideEPCBiguanidesChemicalIngredientPeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredient,PHARM_CLASSES__BileAcidSequestrantEPCBileacidBindingActivityMoA,PHARM_CLASSES__BileAcidEPCBileAcidsandSaltsChemicalIngredient,PHARM_CLASSES__BismuthChemicalIngredientBismuthEPCNitroimidazoleAntimicrobialEPCNitroimidazolesChemicalIngredientTetracyclineclassAntimicrobialEPCTetracyclinesChemicalIngredient,PHARM_CLASSES__BisphosphonateEPCDiphosphonatesChemicalIngredient,PHARM_CLASSES__BloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredient,PHARM_CLASSES__BloodViscosityReducerEPCHematologicActivityAlterationPE,PHARM_CLASSES__CCR5CoreceptorAntagonistEPCChemokineCoreceptor5AntagonistsMoA,PHARM_CLASSES__CalcineurinInhibitorImmunosuppressantEPCCalcineurinInhibitorsMoA,PHARM_CLASSES__CalcineurinInhibitorImmunosuppressantEPCCalcineurinInhibitorsMoACytochromeP4503A4InhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__CalcitoninChemicalIngredientCalcitoninEPC,PHARM_CLASSES__CalciumChannelAntagonistsMoACalciumChannelBlockerEPC,PHARM_CLASSES__CalciumChannelAntagonistsMoACalciumChannelBlockerEPCCytochromeP4503A4InhibitorsMoACytochromeP4503AInhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredient,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientAngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoADecreasedBloodPressurePE,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4502D6InhibitorsMoACytochromeP4502C8InhibitorsMoACytochromeP4502C19InhibitorsMoA,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientHMGCoAReductaseInhibitorEPCHydroxymethylglutarylCoAReductaseInhibitorsMoA,PHARM_CLASSES__CalciumChannelAntagonistsMoADihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC,PHARM_CLASSES__CalciumsensingReceptorAgonistEPCIncreasedCalciumsensingReceptorSensitivityMoA,PHARM_CLASSES__CannabinoidEPCCannabinoidsChemicalIngredient,PHARM_CLASSES__CarbonicAnhydraseInhibitorEPCCarbonicAnhydraseInhibitorsMoA,PHARM_CLASSES__CarbonicAnhydraseInhibitorEPCCarbonicAnhydraseInhibitorsMoAAdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPC,PHARM_CLASSES__CarbonicAnhydraseInhibitorEPCCarbonicAnhydraseInhibitorsMoASulfonamidesChemicalIngredient,PHARM_CLASSES__CardiacGlycosideEPCCardiacGlycosidesChemicalIngredient,PHARM_CLASSES__CarnitineChemicalIngredientCarnitineAnalogEPC,PHARM_CLASSES__CatecholOMethyltransferaseInhibitorsMoACatecholOMethyltransferaseInhibitorEPC,PHARM_CLASSES__CellDeathInducerEPCIncreasedCellularDeathPE,PHARM_CLASSES__CentralNervousSystemStimulantEPCCentralNervousSystemStimulationPE,PHARM_CLASSES__CentralNervousSystemStimulantEPCCentralNervousSystemStimulationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPECentralNervousSystemStimulantEPCCentralNervousSystemStimulationPE,PHARM_CLASSES__CentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredientFullOpioidAgonistsMoAOpioidAgonistEPC,PHARM_CLASSES__CentralNervousSystemStimulationPEIncreasedSympatheticActivityPESympathomimeticlikeAgentEPC,PHARM_CLASSES__CentrallymediatedMuscleRelaxationPE,PHARM_CLASSES__CentrallymediatedMuscleRelaxationPEMuscleRelaxantEPC,PHARM_CLASSES__CephalosporinAntibacterialEPCCephalosporinsChemicalIngredient,PHARM_CLASSES__ChlorideChannelActivatorEPCChlorideChannelActivatorsMoA,PHARM_CLASSES__CholecalciferolChemicalIngredientVitaminD3AnalogEPC,PHARM_CLASSES__CholinergicAgonistsMoACholinergicMuscarinicAgonistsMoACholinergicReceptorAgonistEPC,PHARM_CLASSES__CholinergicMuscarinicAgonistEPCCholinergicMuscarinicAgonistsMoA,PHARM_CLASSES__CholinergicMuscarinicAgonistsMoACholinergicReceptorAgonistEPC,PHARM_CLASSES__CholinergicMuscarinicAntagonistEPCCholinergicMuscarinicAntagonistsMoA,PHARM_CLASSES__CholinesteraseInhibitorEPCCholinesteraseInhibitorsMoA,PHARM_CLASSES__CollagenasesChemicalIngredientCollagenspecificEnzymeEPC,PHARM_CLASSES__CompetitiveOpioidAntagonistsMoAPartialOpioidAgonistAntagonistEPCPartialOpioidAgonistsMoAOpioidAntagonistEPCOpioidAntagonistsMoA,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoAAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoAAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient,PHARM_CLASSES__CorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoACorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__CyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC,PHARM_CLASSES__CyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPEDecreasedPlateletAggregationPEPlateletAggregationInhibitorEPC,PHARM_CLASSES__CyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCProstaglandinE1AnalogEPCProstaglandinsESyntheticChemicalIngredient,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCCyclooxygenaseInhibitorEPC,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCHistamineH2ReceptorAntagonistsMoAHistamine2ReceptorAntagonistEPC,PHARM_CLASSES__CyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCProtonPumpInhibitorEPCProtonPumpInhibitorsMoACytochromeP4502C19InhibitorsMoA,PHARM_CLASSES__CytochromeP45017A1InhibitorsMoACytochromeP45017A1InhibitorEPCCytochromeP4502D6InhibitorsMoACytochromeP4502C8InhibitorsMoA,PHARM_CLASSES__CytochromeP4503A4InhibitorsMoAPGlycoproteinInteractionsMoA,PHARM_CLASSES__CytomegalovirusNucleosideAnalogDNAPolymeraseInhibitorEPCDNAPolymeraseInhibitorsMoANucleosideAnalogChemicalIngredientNucleosideAnalogAntiviralEPC,PHARM_CLASSES__DNAPolymeraseInhibitorsMoAHerpesSimplexVirusNucleosideAnalogDNAPolymeraseInhibitorEPCHerpesZosterVirusNucleosideAnalogDNAPolymeraseInhibitorEPCHerpesvirusNucleosideAnalogDNAPolymeraseInhibitorEPCNucleosideAnalogChemicalIngredient,PHARM_CLASSES__DNAPolymeraseInhibitorsMoAHerpesvirusNucleosideAnalogDNAPolymeraseInhibitorEPCNucleosideAnalogChemicalIngredient,PHARM_CLASSES__DecreasedCellWallIntegrityPE,PHARM_CLASSES__DecreasedCellWallSynthesis&amp;RepairPEAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredientPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient,PHARM_CLASSES__DecreasedCellWallSynthesis&amp;RepairPEPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient,PHARM_CLASSES__DecreasedCentralNervousSystemDisorganizedElectricalActivityPE,PHARM_CLASSES__DecreasedCentralNervousSystemDisorganizedElectricalActivityPECytochromeP4503A4InducersMoACytochromeP4502C19InhibitorsMoA,PHARM_CLASSES__DecreasedCentralNervousSystemDisorganizedElectricalActivityPEMoodStabilizerEPCCytochromeP4503A4InducersMoACytochromeP4501A2InducersMoACytochromeP4502B6InducersMoACytochromeP4502C9InducersMoACytochromeP4502C19InducersMoA,PHARM_CLASSES__DecreasedCholesterolAbsorptionPEDietaryCholesterolAbsorptionInhibitorEPC,PHARM_CLASSES__DecreasedCholesterolAbsorptionPEDietaryCholesterolAbsorptionInhibitorEPCHMGCoAReductaseInhibitorEPCHydroxymethylglutarylCoAReductaseInhibitorsMoA,PHARM_CLASSES__DecreasedDNAReplicationPEDecreasedProteinSynthesisPEDecreasedRNAReplicationPEProteinSynthesisInhibitorsMoA,PHARM_CLASSES__DecreasedHistamineReleasePEHistamineH1ReceptorAntagonistsMoAHistamine1ReceptorInhibitorEPC,PHARM_CLASSES__DecreasedHistamineReleasePEMastCellStabilizerEPC,PHARM_CLASSES__DecreasedImmunologicActivityPEmTORInhibitorImmunosuppressantEPCmTORInhibitorsMoAKinaseInhibitorEPCProteinKinaseInhibitorsMoA,PHARM_CLASSES__DecreasedMitosisPE,PHARM_CLASSES__DecreasedMitosisPEMicrotubuleInhibitionPETubulinInhibitingAgentEPC,PHARM_CLASSES__DecreasedPlateletAggregationPEP2Y12PlateletInhibitorEPCP2Y12ReceptorAntagonistsMoACytochromeP4502C8InhibitorsMoA,PHARM_CLASSES__DecreasedPlateletAggregationPEPlateletAggregationInhibitorEPC,PHARM_CLASSES__DecreasedProteinSynthesisPEDecreasedSebaceousGlandActivityPE,PHARM_CLASSES__DecreasedRenalK+ExcretionPEIncreasedDiuresisPEPotassiumsparingDiureticEPC,PHARM_CLASSES__DecreasedRenalK+ExcretionPEIncreasedDiuresisPEPotassiumsparingDiureticEPCIncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__DecreasedRespiratorySecretionViscosityPEDeoxyribonucleaseIChemicalIngredientRecombinantHumanDeoxyribonuclease1EPCRecombinantProteinsChemicalIngredient,PHARM_CLASSES__DecreasedSebaceousGlandActivityPELincosamideAntibacterialEPCLincosamidesChemicalIngredient,PHARM_CLASSES__DecreasedSebaceousGlandActivityPELincosamideAntibacterialEPCLincosamidesChemicalIngredientRetinoidEPCRetinoidsChemicalIngredient,PHARM_CLASSES__DecreasedSebaceousGlandActivityPEMacrolideEPCMacrolideAntimicrobialEPCMacrolidesChemicalIngredient,PHARM_CLASSES__DecreasedStriatedMuscleContractionPEDecreasedStriatedMuscleTonePESkeletalMuscleRelaxantEPC,PHARM_CLASSES__DecreasedTracheobronchialStretchReceptorActivityPENonnarcoticAntitussiveEPC,PHARM_CLASSES__DemulcentEPCDemulcentActivityMoA,PHARM_CLASSES__DepigmentingActivityPEMelaninSynthesisInhibitorEPCMelaninSynthesisInhibitorsMoA,PHARM_CLASSES__DihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoA,PHARM_CLASSES__DihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoAPolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredient,PHARM_CLASSES__DihydropyridineCalciumChannelBlockerEPCDihydropyridinesChemicalIngredientCalciumChannelAntagonistsMoADecreasedBloodPressurePE,PHARM_CLASSES__DipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPC,PHARM_CLASSES__DipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPCBiguanideEPCBiguanidesChemicalIngredient,PHARM_CLASSES__DipeptidylPeptidase4InhibitorsMoADipeptidylPeptidase4InhibitorEPCPeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredient,PHARM_CLASSES__DopamineAgonistsMoANonergotDopamineAgonistEPC,PHARM_CLASSES__DopamineD2AntagonistsMoADopamine2ReceptorAntagonistEPC,PHARM_CLASSES__EpoxideHydrolaseInhibitorsMoA,PHARM_CLASSES__ErgocalciferolsChemicalIngredientProvitaminD2CompoundEPC,PHARM_CLASSES__ErgocalciferolsChemicalIngredientVitaminD2AnalogEPC,PHARM_CLASSES__ErgolinesChemicalIngredientErgotDerivativeEPC,PHARM_CLASSES__ErgotamineDerivativeEPCErgotaminesChemicalIngredient,PHARM_CLASSES__ErythropoiesisstimulatingAgentEPCErythropoietinChemicalIngredientIncreasedErythroidCellProductionPE,PHARM_CLASSES__EsterLocalAnestheticEPCEstersChemicalIngredientLocalAnesthesiaPE,PHARM_CLASSES__EstradiolCongenersChemicalIngredientEstrogenEPCEstrogenReceptorAgonistsMoA,PHARM_CLASSES__EstradiolCongenersChemicalIngredientEstrogenEPCEstrogenReceptorAgonistsMoAInhibitOvumFertilizationPEProgesteroneCongenersChemicalIngredientProgesteroneCongenersChemicalIngredientProgestinEPCProgestincontainingIntrauterineDeviceEPC,PHARM_CLASSES__EstradiolCongenersChemicalIngredientEstrogenEPCEstrogenReceptorAgonistsMoAProgesteroneCongenersChemicalIngredientProgestinEPC,PHARM_CLASSES__EstrogenAgonistAntagonistEPCSelectiveEstrogenReceptorModulatorsMoA,PHARM_CLASSES__EstrogenEPCEstrogenReceptorAgonistsMoAEstrogensConjugated(USP)ChemicalIngredientAndrogenEPCAndrogenReceptorAgonistsMoAAndrostanesChemicalIngredient,PHARM_CLASSES__EstrogenEPCEstrogenReceptorAgonistsMoAProgesteroneCongenersChemicalIngredientProgestinEPC,PHARM_CLASSES__FactorVIIIActivatorEPCIncreasedCoagulationFactorVIIIActivityPEIncreasedCoagulationFactorVIIIConcentrationPEVasopressinAnalogEPCVasopressinsChemicalIngredient,PHARM_CLASSES__FactorXaInhibitorEPCFactorXaInhibitorsMoA,PHARM_CLASSES__FactorXaInhibitorsMoAFactorXaInhibitorEPC,PHARM_CLASSES__FattyAcidsOmega3ChemicalIngredientOmega3FattyAcidEPC,PHARM_CLASSES__FattyAcidsOmega3ChemicalIngredientOmega3FattyAcidEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCVitaminDChemicalIngredientVitaminDEPCCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPE,PHARM_CLASSES__FattyAcidsOmega3ChemicalIngredientOmega3FattyAcidEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCVitaminDChemicalIngredientVitaminDEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__FolateAnalogMetabolicInhibitorEPCFolicAcidMetabolismInhibitorsMoA,PHARM_CLASSES__FolateAnalogEPCFolicAcidChemicalIngredientAnalogsDerivativesChemicalIngredient,PHARM_CLASSES__FolateAnalogEPCFolicAcidChemicalIngredientAnalogsDerivativesChemicalIngredientMethylatingActivityMoAMethylatingAgentEPCAcidifyingActivityMoACalculiDissolutionAgentEPCAnticoagulantEPCDecreasedCoagulationFactorActivityPECalciumChelatingActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPC,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCBarbiturateEPCBarbituratesChemicalIngredientCentralNervousSystemStimulantEPCCentralNervousSystemStimulationPEMethylxanthineEPCXanthinesChemicalIngredientCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPE,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCCyclooxygenaseInhibitorsMoADecreasedProstaglandinProductionPENonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPCPlateletAggregationInhibitorEPCDecreasedPlateletAggregationPE,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCOpioidAntagonistEPCOpioidAntagonistsMoA,PHARM_CLASSES__FullOpioidAgonistsMoAOpioidAgonistEPCPhenothiazineEPCPhenothiazinesChemicalIngredient,PHARM_CLASSES__GABAAAgonistsMoAGABABAgonistsMoAgammaAminobutyricAcidergicAgonistEPC,PHARM_CLASSES__GlinideEPCPotassiumChannelAntagonistsMoA,PHARM_CLASSES__GlycopeptideAntibacterialEPCGlycopeptidesChemicalIngredient,PHARM_CLASSES__GlycosaminoglycanEPCGlycosaminoglycansChemicalIngredient,PHARM_CLASSES__GranulocyteColonyStimulatingFactorChemicalIngredientGranulocyteMacrophageColonyStimulatingFactorChemicalIngredientIncreasedMyeloidCellProductionPELeukocyteGrowthFactorEPC,PHARM_CLASSES__HIVIntegraseInhibitorsMoAHumanImmunodeficiencyVirusIntegraseStrandTransferInhibitorEPCMultidrugandToxinExtrusionTransporter1InhibitorsMoAOrganicCationTransporter2InhibitorsMoA,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503AInhibitorsMoA,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503AInhibitorsMoACytochromeP4502D6InhibitorsMoA,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503AInhibitorsMoACytochromeP4502D6InhibitorsMoACytochromeP4503AInhibitorsMoACytochromeP4503AInhibitorEPCPGlycoproteinInhibitorsMoACytochromeP4502D6InhibitorsMoAOrganicAnionTransportingPolypeptide1B1InhibitorsMoAOrganicAnionTransportingPolypeptide1B3InhibitorsMoABreastCancerResistanceProteinInhibitorsMoA,PHARM_CLASSES__HIVProteaseInhibitorsMoAProteaseInhibitorEPCCytochromeP4503A4InhibitorsMoACytochromeP4503A4InducersMoAPGlycoproteinInducersMoA,PHARM_CLASSES__HMGCoAReductaseInhibitorEPCHydroxymethylglutarylCoAReductaseInhibitorsMoA,PHARM_CLASSES__HeparinLowMolecularWeightChemicalIngredientLowMolecularWeightHeparinEPC,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HistamineH1ReceptorAntagonistsMoAHistamine1ReceptorAntagonistEPC,PHARM_CLASSES__HistamineH1ReceptorAntagonistsMoAHistamine1ReceptorAntagonistEPCAdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPC,PHARM_CLASSES__HistamineH2ReceptorAntagonistsMoAHistamine2ReceptorAntagonistEPC,PHARM_CLASSES__Histamine1ReceptorAntagonistEPC,PHARM_CLASSES__HumanImmunodeficiencyVirus1NonNucleosideAnalogReverseTranscriptaseInhibitorEPCNonNucleosideAnalogChemicalIngredientNonNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HumanImmunodeficiencyVirus1NonNucleosideAnalogReverseTranscriptaseInhibitorEPCNonNucleosideAnalogChemicalIngredientNonNucleosideReverseTranscriptaseInhibitorsMoACytochromeP4503AInducersMoACytochromeP4502B6InducersMoA,PHARM_CLASSES__HumanImmunodeficiencyVirus1NonNucleosideAnalogReverseTranscriptaseInhibitorEPCNonNucleosideReverseTranscriptaseInhibitorsMoANonNucleosideAnalogChemicalIngredient,PHARM_CLASSES__HumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHIVIntegraseInhibitorsMoAHumanImmunodeficiencyVirusIntegraseStrandTransferInhibitorEPCMultidrugandToxinExtrusionTransporter1InhibitorsMoAOrganicCationTransporter2InhibitorsMoAHepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoAHepatitisBVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCHumanImmunodeficiencyVirusNucleosideAnalogReverseTranscriptaseInhibitorEPCNucleosideAnalogChemicalIngredientNucleosideReverseTranscriptaseInhibitorsMoA,PHARM_CLASSES__HumanPlateletderivedGrowthFactorEPCPlateletDerivedGrowthFactorChemicalIngredient,PHARM_CLASSES__HyperpolarizationactivatedCyclicNucleotidegatedChannelBlockerEPCHyperpolarizationactivatedCyclicNucleotidegatedChannelAntagonistsMoA,PHARM_CLASSES__IncreasedCytokineActivityPEIncreasedCytokineProductionPEInterferonInducersMoA,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredient,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAdrenergicbetaAntagonistsMoAbetaAdrenergicBlockerEPC,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAngiotensin2ReceptorAntagonistsMoAAngiotensin2ReceptorBlockerEPC,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientAngiotensinConvertingEnzymeInhibitorEPCAngiotensinconvertingEnzymeInhibitorsMoA,PHARM_CLASSES__IncreasedDiuresisPEThiazideDiureticEPCThiazidesChemicalIngredientDecreasedRenalK+ExcretionPEIncreasedDiuresisPEPotassiumsparingDiureticEPC,PHARM_CLASSES__IncreasedDiuresisPEThiazidelikeDiureticEPC,PHARM_CLASSES__IncreasedDiuresisatLoopofHenlePELoopDiureticEPC,PHARM_CLASSES__InfluenzaAM2ProteinInhibitorEPCM2ProteinInhibitorsMoA,PHARM_CLASSES__InhibitOvumFertilizationPEProgesteroneCongenersChemicalIngredientProgesteroneCongenersChemicalIngredientProgestinEPCProgestincontainingIntrauterineDeviceEPCEstrogenEPCEstrogenReceptorAgonistsMoA,PHARM_CLASSES__Interleukin12AntagonistEPCInterleukin12AntagonistsMoAInterleukin23AntagonistEPCInterleukin23AntagonistsMoA,PHARM_CLASSES__IronChemicalIngredientParenteralIronReplacementEPCPhosphateBinderEPCPhosphateChelatingActivityMoA,PHARM_CLASSES__KinaseInhibitorEPCProteinKinaseInhibitorsMoA,PHARM_CLASSES__LeukotrieneReceptorAntagonistEPCLeukotrieneReceptorAntagonistsMoA,PHARM_CLASSES__LeukotrieneReceptorAntagonistEPCLeukotrieneReceptorAntagonistsMoACytochromeP4502C9InhibitorsMoA,PHARM_CLASSES__LocalAnesthesiaPELocalAnestheticEPC,PHARM_CLASSES__LymphocyteFunctionAssociatedAntigen1AntagonistEPCLymphocyteFunctionAssociatedAntigen1AntagonistsMoA,PHARM_CLASSES__MacrolideAntimicrobialEPCMacrolidesChemicalIngredient,PHARM_CLASSES__MacrolideAntimicrobialEPCMacrolidesChemicalIngredientCytochromeP4503A4InhibitorsMoACytochromeP4503AInhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__MacrolidesChemicalIngredientMacrolideAntibacterialEPC,PHARM_CLASSES__MelatoninReceptorAgonistEPCMelatoninReceptorAgonistsMoA,PHARM_CLASSES__MethylxanthineEPCXanthinesChemicalIngredient,PHARM_CLASSES__MonoamineOxidaseInhibitorEPCMonoamineOxidaseInhibitorsMoA,PHARM_CLASSES__MonoamineOxidaseInhibitorEPCMonoamineOxidaseInhibitorsMoAMonoamineOxidaseTypeBInhibitorEPCMonoamineOxidaseBInhibitorsMoA,PHARM_CLASSES__MoodStabilizerEPC,PHARM_CLASSES__NMDAReceptorAntagonistsMoANmethylDaspartateReceptorAntagonistEPC,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredient,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPECopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE,PHARM_CLASSES__NicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE,PHARM_CLASSES__NitrateVasodilatorEPCNitratesChemicalIngredientVasodilationPE,PHARM_CLASSES__NitrofuranAntibacterialEPCNitrofuransChemicalIngredient,PHARM_CLASSES__NitrofuranAntibacterialEPCNitrofuransChemicalIngredientNitrofuranAntibacterialEPCNitrofuransChemicalIngredient,PHARM_CLASSES__NitroimidazoleAntimicrobialEPCNitroimidazolesChemicalIngredient,PHARM_CLASSES__NoncompetitiveAMPAGlutamateReceptorAntagonistEPCAMPAReceptorAntagonistsMoA,PHARM_CLASSES__NorepinephrineUptakeInhibitorsMoASerotoninandNorepinephrineReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoA,PHARM_CLASSES__NorepinephrineUptakeInhibitorsMoASerotoninandNorepinephrineReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoACytochromeP4502D6InhibitorsMoA,PHARM_CLASSES__NucleicAcidSynthesisInhibitorsMoANucleosideAnalogChemicalIngredientPurineAntimetaboliteEPCPurinesChemicalIngredient,PHARM_CLASSES__NucleicAcidSynthesisInhibitorsMoANucleosideMetabolicInhibitorEPC,PHARM_CLASSES__NucleosideAnalogChemicalIngredientNucleosideAnalogAntiviralEPC,PHARM_CLASSES__NucleosideAnalogChemicalIngredientNucleosideAnalogAntiviralEPCNucleosideMetabolicInhibitorEPCNucleicAcidSynthesisInhibitorsMoA,PHARM_CLASSES__OpioidAgonistEPCOpioidAgonistsMoA,PHARM_CLASSES__OpioidAgonistEPCOpioidAgonistsMoACyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC,PHARM_CLASSES__OpioidAgonistEPCOpioidAgonistsMoAHistamineH1ReceptorAntagonistsMoAHistamine1ReceptorAntagonistEPC,PHARM_CLASSES__OpioidAntagonistEPCOpioidAntagonistsMoA,PHARM_CLASSES__OpioidAntagonistEPCOpioidAntagonistsMoAAminoketoneEPCDopamineUptakeInhibitorsMoAIncreasedDopamineActivityPEIncreasedNorepinephrineActivityPENorepinephrineUptakeInhibitorsMoA,PHARM_CLASSES__OpioidAntagonistEPCOpioidAntagonistsMoAQuaternaryAmmoniumCompoundsChemicalIngredient,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPE,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoACalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAOsmoticActivityMoAOsmoticLaxativeEPCStimulationLargeIntestineFluidElectrolyteSecretionPEOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__OsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoAVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE,PHARM_CLASSES__OsmoticLaxativeEPCOsmoticActivityMoAAcidifyingActivityMoAStimulationLargeIntestineFluidElectrolyteSecretionPE,PHARM_CLASSES__OxaboroleAntifungalEPCProteinSynthesisInhibitorsMoABoronCompoundsChemicalIngredient,PHARM_CLASSES__OxazolidinoneAntibacterialEPCOxazolidinonesChemicalIngredient,PHARM_CLASSES__PCSK9InhibitorEPCPCSK9InhibitorsMoAAntibodiesMonoclonalChemicalIngredient,PHARM_CLASSES__PartialOpioidAgonistEPCPartialOpioidAgonistsMoA,PHARM_CLASSES__PartialOpioidAgonistEPCPartialOpioidAgonistsMoAOpioidAntagonistEPCOpioidAntagonistsMoA,PHARM_CLASSES__PediculicideEPC,PHARM_CLASSES__PenicillinclassAntibacterialEPCPenicillinsChemicalIngredient,PHARM_CLASSES__PenicillinclassAntibacterialEPCPenicillinsChemicalIngredientbetaLactamaseInhibitorEPCbetaLactamaseInhibitorsMoA,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPC,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredient,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredientBiguanideEPCBiguanidesChemicalIngredient,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatorReceptorgammaAgonistEPCPeroxisomeProliferatoractivatedReceptorActivityMoAPPARalphaChemicalIngredientPPARgammaChemicalIngredientThiazolidinedioneEPCThiazolidinedionesChemicalIngredientSulfonylureaEPCSulfonylureaCompoundsChemicalIngredient,PHARM_CLASSES__PeroxisomeProliferatorReceptoralphaAgonistEPCPeroxisomeProliferatoractivatedReceptoralphaAgonistsMoAPPARalphaChemicalIngredient,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredient,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientFullOpioidAgonistsMoAOpioidAgonistEPC,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientUncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoA,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientalpha1AdrenergicAgonistEPCAdrenergicalpha1AgonistsMoA,PHARM_CLASSES__PhenothiazineEPCPhenothiazinesChemicalIngredientalpha1AdrenergicAgonistEPCAdrenergicalpha1AgonistsMoAFullOpioidAgonistsMoAOpioidAgonistEPC,PHARM_CLASSES__PhosphateBinderEPCPhosphateChelatingActivityMoA,PHARM_CLASSES__Phosphodiesterase3InhibitorEPCPhosphodiesterase3InhibitorsMoA,PHARM_CLASSES__Phosphodiesterase4InhibitorEPCPhosphodiesterase4InhibitorsMoA,PHARM_CLASSES__Phosphodiesterase5InhibitorEPCPhosphodiesterase5InhibitorsMoA,PHARM_CLASSES__PolyeneAntifungalEPCPolyenesChemicalIngredient,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoAAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDecreasedCellWallSynthesis&amp;RepairPE,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDecreasedCellWallSynthesis&amp;RepairPEAminoglycosideAntibacterialEPCAminoglycosidesChemicalIngredient,PHARM_CLASSES__PolymyxinclassAntibacterialEPCPolymyxinsChemicalIngredientDihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoA,PHARM_CLASSES__PotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__ProgesteroneCongenersChemicalIngredientProgestinEPC,PHARM_CLASSES__ProgesteroneCongenersChemicalIngredientProgestinEPCEstrogenEPCEstrogenReceptorAgonistsMoA,PHARM_CLASSES__ProgesteroneChemicalIngredientProgesteroneEPC,PHARM_CLASSES__ProstaglandinAnalogEPCProstaglandinReceptorAgonistsMoAIncreasedProstaglandinActivityPEProstaglandinsChemicalIngredient,PHARM_CLASSES__ProstaglandinAnalogEPCProstaglandinsChemicalIngredient,PHARM_CLASSES__ProstaglandinE1AnalogEPCProstaglandinsESyntheticChemicalIngredient,PHARM_CLASSES__ProteaseactivatedReceptor1AntagonistEPCProteaseactivatedReceptor1AntagonistsMoA,PHARM_CLASSES__ProteasomeInhibitorEPCProteasomeInhibitorsMoA,PHARM_CLASSES__ProtonPumpInhibitorEPCProtonPumpInhibitorsMoA,PHARM_CLASSES__ProtonPumpInhibitorEPCProtonPumpInhibitorsMoACytochromeP4502C19InhibitorsMoA,PHARM_CLASSES__ProtonPumpInhibitorEPCProtonPumpInhibitorsMoAInhibitionGastricAcidSecretionPE,PHARM_CLASSES__PyrethrinsChemicalIngredientPyrethroidEPC,PHARM_CLASSES__PyrimidineSynthesisInhibitorEPCDihydroorotateDehydrogenaseInhibitorsMoA,PHARM_CLASSES__QuinoloneAntimicrobialEPCQuinolonesChemicalIngredient,PHARM_CLASSES__QuinoloneAntimicrobialEPCQuinolonesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__RANKLigandInhibitorEPCRANKLigandBlockingActivityMoA,PHARM_CLASSES__RNASynthetaseInhibitorAntibacterialEPCRNASynthetaseInhibitorsMoA,PHARM_CLASSES__ReninInhibitorEPCReninInhibitorsMoA,PHARM_CLASSES__RetinoidEPCRetinoidsChemicalIngredient,PHARM_CLASSES__RifamycinAntibacterialEPCRifamycinsChemicalIngredient,PHARM_CLASSES__Serotonin1bReceptorAgonistsMoASerotonin1dReceptorAgonistsMoASerotonin1bandSerotonin1dReceptorAgonistEPC,PHARM_CLASSES__Serotonin1bReceptorAgonistsMoASerotonin1dReceptorAgonistsMoASerotonin1bandSerotonin1dReceptorAgonistEPCCyclooxygenaseInhibitorsMoANonsteroidalAntiinflammatoryCompoundsChemicalIngredientNonsteroidalAntiinflammatoryDrugEPC,PHARM_CLASSES__Serotonin3ReceptorAntagonistsMoASerotonin3ReceptorAntagonistEPC,PHARM_CLASSES__SerotoninReuptakeInhibitorEPC,PHARM_CLASSES__SerotoninReuptakeInhibitorEPCSerotoninUptakeInhibitorsMoA,PHARM_CLASSES__Serotonin2cReceptorAgonistEPCSerotonin2cReceptorAgonistsMoACytochromeP4502D6InhibitorsMoA,PHARM_CLASSES__SkinTestAntigenEPCTuberculosisSkinTestEPCCellmediatedImmunityPEAntigensBacterialChemicalIngredient,PHARM_CLASSES__SodiumGlucoseCotransporter2InhibitorEPCSodiumGlucoseTransporter2InhibitorsMoAPGlycoproteinInhibitorsMoA,PHARM_CLASSES__SodiumGlucoseCotransporter2InhibitorEPCSodiumGlucoseTransporter2InhibitorsMoAPGlycoproteinInhibitorsMoABiguanideEPCBiguanidesChemicalIngredient,PHARM_CLASSES__SomatostatinAnalogEPCSomatostatinReceptorAgonistsMoA,PHARM_CLASSES__SulfonamideAntibacterialEPCSulfonamidesChemicalIngredient,PHARM_CLASSES__SulfonamideAntibacterialEPCSulfonamidesChemicalIngredientCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__SulfonamideAntimicrobialEPCSulfonamidesChemicalIngredientCytochromeP4502C9InhibitorsMoADihydrofolateReductaseInhibitorAntibacterialEPCDihydrofolateReductaseInhibitorsMoACytochromeP4502C8InhibitorsMoAOrganicCationTransporter2InhibitorsMoA,PHARM_CLASSES__SulfoneEPCSulfonesChemicalIngredient,PHARM_CLASSES__SulfonylureaEPCSulfonylureaCompoundsChemicalIngredient,PHARM_CLASSES__SulfonylureaEPCSulfonylureaCompoundsChemicalIngredientBiguanideEPCBiguanidesChemicalIngredient,PHARM_CLASSES__TetracyclineclassAntimicrobialEPCTetracyclinesChemicalIngredient,PHARM_CLASSES__TetracyclineclassDrugEPCTetracyclinesChemicalIngredient,PHARM_CLASSES__ThyroidHormoneSynthesisInhibitorEPCThyroidHormoneSynthesisInhibitorsMoA,PHARM_CLASSES__TricyclicAntidepressantEPC,PHARM_CLASSES__TumorNecrosisFactorBlockerEPCTumorNecrosisFactorReceptorBlockingActivityMoA,PHARM_CLASSES__TypicalAntipsychoticEPC,PHARM_CLASSES__UncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoAAdrenergicalphaAgonistsMoAalphaAdrenergicAgonistEPC,PHARM_CLASSES__UncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoAAntiarrhythmicEPCCytochromeP4502D6InhibitorEPCCytochromeP4502D6InhibitorsMoA,PHARM_CLASSES__UncompetitiveNmethylDaspartateReceptorAntagonistEPCSigma1AgonistEPCUncompetitiveNMDAReceptorAntagonistsMoASigma1ReceptorAgonistsMoAPhenothiazineEPCPhenothiazinesChemicalIngredient,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPC,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminB12ChemicalIngredientVitaminB12EPCVitaminDChemicalIngredientVitaminDEPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCIronChemicalIngredientParenteralIronReplacementEPCPhosphateBinderEPCPhosphateChelatingActivityMoA,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB12ChemicalIngredientVitaminB12EPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE,PHARM_CLASSES__VitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__VitaminB12ChemicalIngredientVitaminB12EPC,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientVitaminDChemicalIngredientVitaminDEPCVitaminB12ChemicalIngredientVitaminB12EPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPE,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPC,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPC,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCopperChemicalIngredientCoppercontainingIntrauterineDeviceEPCDecreasedEmbryonicImplantationPEDecreasedSpermMotilityPEInhibitOvumFertilizationPE,PHARM_CLASSES__VitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCVitaminAChemicalIngredientVitaminAEPCVitaminDChemicalIngredientVitaminDEPC,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDAnalogEPC,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDAnalogEPCCorticosteroidEPCCorticosteroidHormoneReceptorAgonistsMoA,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCNicotinicAcidEPCNicotinicAcidsChemicalIngredientVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientAntidoteEPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB12ChemicalIngredientVitaminB12EPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredient,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCBloodCoagulationFactorEPCIncreasedCoagulationFactorActivityPECalciumChemicalIngredientCationsDivalentChemicalIngredientCopperAbsorptionInhibitorEPCDecreasedCopperIonAbsorptionPE,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCalciumChemicalIngredientCalciumEPCPhosphateBinderEPCPhosphateChelatingActivityMoAPotassiumCompoundsChemicalIngredientPotassiumSaltEPCOsmoticLaxativeEPCIncreasedLargeIntestinalMotilityPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPEOsmoticActivityMoA,PHARM_CLASSES__VitaminDChemicalIngredientVitaminDEPCVitaminB6AnalogEPCVitaminB6ChemicalIngredientAnalogsDerivativesChemicalIngredientVitaminB12ChemicalIngredientVitaminB12EPCCalculiDissolutionAgentEPCMagnesiumIonExchangeActivityMoAOsmoticLaxativeEPCOsmoticActivityMoAInhibitionSmallIntestineFluidElectrolyteAbsorptionPEIncreasedLargeIntestinalMotilityPEStimulationLargeIntestineFluidElectrolyteSecretionPEInhibitionLargeIntestineFluidElectrolyteAbsorptionPE,PHARM_CLASSES__VitaminKAntagonistEPCVitaminKInhibitorsMoA,PHARM_CLASSES__XanthineOxidaseInhibitorEPCXanthineOxidaseInhibitorsMoA,PHARM_CLASSES__alphaGlucosidaseInhibitorsMoAalphaGlucosidaseInhibitorEPC,PHARM_CLASSES__gammaAminobutyricAcidAReceptorAgonistEPCGABAAAgonistsMoACentralNervousSystemDepressionPE,PHARM_CLASSES__gammaAminobutyricAcidergicAgonistEPCGABAAAgonistsMoAPyridinesChemicalIngredientCentralNervousSystemDepressionPE,PHARM_CLASSES__lThyroxineEPCThyroxineChemicalIngredient,PHARM_CLASSES__lThyroxineEPCThyroxineChemicalIngredientlTriiodothyronineEPCTriiodothyronineChemicalIngredient,PHARM_CLASSES__lTriiodothyronineEPCTriiodothyronineChemicalIngredient,PHARM_CLASSES__muOpioidReceptorAgonistEPCOpioidmuReceptorAgonistsMoA,priceType)

SyntaxError: invalid syntax (<ipython-input-51-4aebabacabdb>, line 1)